In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels as sm
import datetime
from itertools import compress
import os
import warnings
warnings.filterwarnings('ignore')
import xlrd
from pandas import ExcelWriter
import requests
import streamlit as st
import matplotlib.backends.backend_pdf
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import plotly.graph_objects as go
import matplotlib.colors as colors


IMPORTACIÓ DADES

In [2]:
# os_name ="linux"
os_name = "windows"

if os_name=="linux":
    path = "/home/ruben/Dropbox/Estudi d'oferta/2023/1S 2023/"
    path_aux = "/home/ruben/Dropbox/Estudi d'oferta/"
else:
    user="joana.APCE" #joana.APCE
    path = "C:/Users/" + user + "/Dropbox/Estudi d'oferta/2023/1S 2023/"
    path_aux = "C:/Users/joana.APCE/Dropbox/Estudi d'oferta/"


maestro_estudi = pd.read_excel(path_aux + "Maestro estudi_oferta.xlsx", sheet_name="Maestro").rename(columns={"GEO":"Municipi"})

def tidy_bbdd(any):
    # Importar BBDD promocions d'habitatge
    bbdd_estudi_prom = pd.read_excel(path + 'BBDD 1S2023 15.06.23.xlsx', sheet_name='Promocions 2023')
    bbdd_estudi_prom.columns = bbdd_estudi_prom.iloc[0,:]
    bbdd_estudi_prom = bbdd_estudi_prom[bbdd_estudi_prom["ESTUDI"]==any]
    bbdd_estudi_prom['TIPO_aux'] = np.where(bbdd_estudi_prom['TIPO'].isin([1,2]), 'Habitatges Unifamiliars', 'Habitatges Plurifamiliars')

    mapping = {1: 'Unifamiliars aïllats', 
            2: 'Unifamiliars adossats', 
            3: 'Plurifamiliars en bloc obert', 
            4: 'Plurifamiliars en bloc tancat'}

    mapping1 = {1: "De nova Construcció",
                2: "Rehabilitació integral"}

    mapping2 = {1: "Pendent d'enderroc", 
            2: "Solar", 
            3: "Buidat", 
            4: "Cimentació",
            5: "Estructura",
            6: "Tancaments exteriors",
            7: "Tancaments interiors",
            8: "Claus en mà",
            9: "NS/NC"}

    mapping3 = {
                    1: 'A',
                    1.2:"A",
                    2: 'B',
                    2.3: "B",
                    3: 'C',
                    4: 'D',
                    4.5: "D",
                    5: 'E',
                    5.3 : "C",
                    6: "F",
                    7: "G",
                    8: "En tràmits",
                    9: "Sense informació"
    }

    mapping4 = {
                    0: "Altres",
                    1: "Plaça d'aparcament opcional",
                    2: "Plaça d'aparcament inclosa",
                    3: "Sense plaça d'aparcament",
    }


    # bbdd_estudi_hab['QENERGC'] = bbdd_estudi_hab['QENERGC'].map(number_to_letter_map)

    bbdd_estudi_prom['TIPO'] = bbdd_estudi_prom['TIPO'].map(mapping)

    bbdd_estudi_prom['TIPH'] = bbdd_estudi_prom['TIPH'].map(mapping1)


    bbdd_estudi_prom['ESTO'] = bbdd_estudi_prom['ESTO'].map(mapping2)

    bbdd_estudi_prom['QENERGC'] = bbdd_estudi_prom['QENERGC'].map(mapping3)

    bbdd_estudi_prom['APAR'] = bbdd_estudi_prom['APAR'].map(mapping4)


    # Importar BBDD habitatges
    bbdd_estudi_hab = pd.read_excel(path + 'BBDD 1S2023 15.06.23.xlsx', sheet_name='Habitatges 2023')
    bbdd_estudi_hab.columns = bbdd_estudi_hab.iloc[0,:]
    bbdd_estudi_hab = bbdd_estudi_hab[bbdd_estudi_hab["ESTUDI"]==any]





    # ["Total dormitoris","Banys i lavabos","Cuines estàndard","Cuines americanes","Terrasses, balcons i patis","Estudi/golfes","Safareig","Altres interiors","Altres exteriors"]

    # ["DORM", "LAV", "cuina_normal", "cuina_amer", "TER", "Golfes", "Safareig","Altres interiors","Altres exteriors" ]

    bbdd_estudi_hab['TIPOG'] = np.where(bbdd_estudi_hab['TIPO'].isin([1,2]), 'Habitatges Unifamiliars', 'Habitatges Plurifamiliars')
    bbdd_estudi_hab['TIPO'] = bbdd_estudi_hab['TIPO'].map(mapping)
    bbdd_estudi_hab['QENERGC'] = bbdd_estudi_hab['QENERGC'].map(mapping3)
    bbdd_estudi_hab['APAR'] = bbdd_estudi_hab['APAR'].map(mapping4)

    bbdd_estudi_hab = bbdd_estudi_hab.dropna(axis=1 , how ='all')



    bbdd_estudi_hab = bbdd_estudi_hab.rename(columns = {'V0006':'Total dormitoris_aux', 
                                                            "DORM": "Total dormitoris",
                                                            "LAV": "Banys i lavabos",
                                                            "TER": "Terrasses, balcons i patis",
                                                            'NOMD01C':'Superfície útil',
                                                            "Preu_m2_util": "Preu m2 útil",
                                                            "NOMD01F_2022": "Preu mitjà",
                                                            'NOMD01P':'Estudi/golfes', 
                                                            'NOMD01Q':'Safareig', 
                                                            'NOMD01K': 'Cuines estàndard', 
                                                            'NOMD01L': 'Cuines americanes', 
                                                            "NOMD01R": "Altres interiors", 
                                                            "NOMD01S":"Altres exteriors"})

    bbdd_estudi_prom = bbdd_estudi_prom.rename(columns = {'V0006':'Total dormitoris_aux', 
                                                            "DORM": "Total dormitoris",
                                                            "LAV": "Banys i lavabos",
                                                            "TER": "Terrasses, balcons i patis",
                                                            'NOMD01C':'Superfície útil',
                                                            "Preu_m2_util": "Preu m2 útil",
                                                            'NOMD01P':'Estudi/golfes', 
                                                            'NOMD01Q':'Safareig', 
                                                            'NOMD01K': 'Cuines estàndard', 
                                                            'NOMD01L': 'Cuines americanes', 
                                                            "NOMD01R": "Altres interiors", 
                                                            "NOMD01S":"Altres exteriors"})


    # Canviar de nom tots els equipaments
    bbdd_estudi_hab = bbdd_estudi_hab.rename(columns = {'EQUIC_1': 'Zona enjardinada', 
                                                        'EQUIC_2': 'Parc infantil',
                                                        'EQUIC_3': 'Piscina comunitària', 
                                                        'EQUIC_4': 'Traster', 
                                                        'EQUIC_5': 'Ascensor', 
                                                        'EQUIC_6': 'Equipament Esportiu',  
                                                        'EQUIC_7': 'Sala de jocs', 
                                                        'EQUIC_8': 'Sauna', 
                                                        "EQUIC_9_50": "Altres",
                                                        'EQUIC_99': 'Cap dels anteriors'})
    bbdd_estudi_prom = bbdd_estudi_prom.rename(columns = {'EQUIC_1': 'Zona enjardinada', 
                                                        'EQUIC_2': 'Parc infantil',
                                                        'EQUIC_3': 'Piscina comunitària', 
                                                        'EQUIC_4': 'Traster', 
                                                        'EQUIC_5': 'Ascensor', 
                                                        'EQUIC_6': 'Equipament Esportiu',  
                                                        'EQUIC_7': 'Sala de jocs', 
                                                        'EQUIC_8': 'Sauna', 
                                                        "QUAL_ALTRES": "Altres",
                                                        'EQUIC_99': 'Cap dels anteriors'})
    bbdd_estudi_prom["Ascensor"] = np.where(bbdd_estudi_prom["Ascensor"]>=1, 1, bbdd_estudi_prom["Ascensor"])
    bbdd_estudi_hab["Ascensor"] = np.where(bbdd_estudi_hab["Ascensor"]>=1, 1, bbdd_estudi_hab["Ascensor"])


    # Canviar de nom totes les qualitats
    bbdd_estudi_hab = bbdd_estudi_hab.rename(columns = {"QUALIC_5": "Aire condicionat", 
                                                        "QUALIC_6": "Bomba de calor", 
                                                        "QUALI_A": "Aero", 
                                                        'QUALIC_7':"Calefacció", 
                                                        'QUALIC_8':"Preinstal·lació d'A.C./B. Calor/Calefacció", 
                                                        'QUALIC_9': 'Parquet', 
                                                        'QUALIC_10':'Armaris encastats',
                                                        'QUALIC_12':'Placa de cocció amb gas',
                                                        'QUALIC_13':'Placa de cocció vitroceràmica',
                                                        "QUALIC_14":"Placa d'inducció",
                                                        'QUALIC_22':'Plaques solars'})


    bbdd_estudi_prom = bbdd_estudi_prom.rename(columns = {"QUALIC_5": "Aire condicionat", 
                                                        "QUALIC_6": "Bomba de calor", 
                                                        "QUALI_A": "Aero", 
                                                        'QUALIC_7':"Calefacció", 
                                                        'QUALIC_8':"Preinstal·lació d'A.C./B. Calor/Calefacció", 
                                                        'QUALIC_9': 'Parquet', 
                                                        'QUALIC_10':'Armaris encastats',
                                                        'QUALIC_12':'Placa de cocció amb gas',
                                                        'QUALIC_13':'Placa de cocció vitroceràmica',
                                                        "QUALIC_14":"Placa d'inducció",
                                                        'QUALIC_22':'Plaques solars'})
    #  Canviar nom a tipus de calefacció
    bbdd_estudi_prom = bbdd_estudi_prom.rename(columns = {'CALEFC_3': 'De gasoil', 
                                                        'CALEFC_4': 'De gas natural', 
                                                        'CALEFC_5': 'De propà', 
                                                        'CALEFC_6': "D'electricitat", 
                                                        'CALEFC_9': "No s'indica tipus"})




    bbdd_estudi_prom['TIPV'] = np.where(bbdd_estudi_prom['TIPV_1'] >= 1, "Venda a través d'immobiliària independent",
                                        np.where(bbdd_estudi_prom['TIPV_2'] >= 1, "Venda a través d'immobiliaria del mateix promotor",
                                                np.where(bbdd_estudi_prom['TIPV_3'] >= 1, "Venda directa del promotor", "Sense informació")))


    bbdd_estudi_prom['TIPOL_VENDA'] = np.where(bbdd_estudi_prom['TIPOL_VENDA_1'] == 1, "0D",
                                        np.where(bbdd_estudi_prom['TIPOL_VENDA_2'] == 1, "1D",
                                                np.where(bbdd_estudi_prom['TIPOL_VENDA_3'] == 1, "2D",
                                                        np.where(bbdd_estudi_prom['TIPOL_VENDA_4'] == 1, "3D",
                                                            np.where(bbdd_estudi_prom['TIPOL_VENDA_5'] == 1, "4D", 
                                                                np.where(bbdd_estudi_prom['TIPOL_VENDA_6'] == 1, "5+D", "NA"))))))

                        
                                                    
    #  "Venda a través d'immobiliària independent", "Venda a través d'immobiliaria del mateix promotor", "Venda directa del promotor"

    bbdd_estudi_hab['TIPH'] = bbdd_estudi_hab['TIPH'].map(mapping1)

    bbdd_estudi_hab['ESTO'] = bbdd_estudi_hab['ESTO'].map(mapping2)


    vars = ['Zona enjardinada', 'Parc infantil', 'Piscina comunitària', 
            'Traster', 'Ascensor', 'Equipament Esportiu', 'Sala de jocs', 
            'Sauna', 'Altres', "Aire condicionat", "Bomba de calor", 
            "Aero", "Calefacció", "Preinstal·lació d'A.C./B. Calor/Calefacció", 
            "Parquet", "Armaris encastats", 'Placa de cocció amb gas', 
            'Placa de cocció vitroceràmica', "Placa d'inducció", 'Plaques solars', "APAR"]
    vars_aux = ['Zona enjardinada', 'Parc infantil', 'Piscina comunitària', 
            'Traster', 'Ascensor', 'Equipament Esportiu', 'Sala de jocs', 
            'Sauna', 'Altres', "Aire condicionat", "Bomba de calor", 
            "Aero", "Calefacció", "Preinstal·lació d'A.C./B. Calor/Calefacció", 
            "Parquet", "Armaris encastats", 'Placa de cocció amb gas', 
            'Placa de cocció vitroceràmica', "Placa d'inducció", 'Plaques solars', "Safareig","Terrasses, balcons i patis"]
    for i in vars:
        bbdd_estudi_prom[i] = bbdd_estudi_prom[i].replace(np.nan, 0)
    for i in vars_aux:
        bbdd_estudi_hab[i] = bbdd_estudi_hab[i].replace(np.nan, 0)
    bbdd_estudi_hab["Calefacció"] = bbdd_estudi_hab["Calefacció"].replace(' ', 0) 
    bbdd_estudi_prom["Calefacció"] = bbdd_estudi_prom["Calefacció"].replace(' ', 0) 


    bbdd_estudi_hab["Tram_Sup_util"] = bbdd_estudi_hab["Tram_Sup_util"].str.replace(" ", "")
    bbdd_estudi_hab["Tram_Sup_util"] = bbdd_estudi_hab["Tram_Sup_util"].str[3:]



    # Afegir categories a algunes columnes de la base de dades d'habitatge

    room_dict =  {i: f"{i}D" if i <= 4 else "5+D" for i in range(0, 20)}
    toilet_dict = {i: f"{i} Bany" if i <= 1 else "2 i més Banys" for i in range(1, 20)}
    bbdd_estudi_hab_mod = bbdd_estudi_hab.copy()

    bbdd_estudi_hab_mod['Total dormitoris'] = bbdd_estudi_hab_mod['Total dormitoris'].map(room_dict)
    bbdd_estudi_hab_mod['Banys i lavabos'] = bbdd_estudi_hab_mod['Banys i lavabos'].map(toilet_dict)
    bbdd_estudi_hab_mod["Terrasses, balcons i patis"] = np.where(bbdd_estudi_hab_mod["Terrasses, balcons i patis"]>=1, 1, 0)

    return([bbdd_estudi_prom, bbdd_estudi_hab, bbdd_estudi_hab_mod])



bbdd_estudi_prom, bbdd_estudi_hab, bbdd_estudi_hab_mod = tidy_bbdd(2023)

In [3]:
bbdd_estudi_prom[bbdd_estudi_prom["Nom BARRI"]=='33 el Baix Guinardó\xa0\xa0']

bbdd_estudi_prom["Nom BARRI"] = np.where(bbdd_estudi_prom["Nom BARRI"] =='33 el Baix Guinardó\xa0\xa0', "33 el Baix Guinardó", bbdd_estudi_prom["Nom BARRI"])
bbdd_estudi_prom["Nom BARRI"] = np.where(bbdd_estudi_prom["Nom BARRI"] =='45 Porta\xa0\xa0', "45 Porta", bbdd_estudi_prom["Nom BARRI"])

In [4]:
def tidy_bbdd(any):
    # Importar BBDD promocions d'habitatge
    bbdd_estudi_prom = pd.read_excel(path + 'BBDD 2022_2021 03.02.23.xlsx', sheet_name='Promocions 2022_2021')
    bbdd_estudi_prom.columns = bbdd_estudi_prom.iloc[0,:]
    bbdd_estudi_prom = bbdd_estudi_prom[bbdd_estudi_prom["ESTUDI"]==any]
    bbdd_estudi_prom['TIPO_aux'] = np.where(bbdd_estudi_prom['TIPO'].isin([1,2]), 'Habitatges Unifamiliars', 'Habitatges Plurifamiliars')

    mapping = {1: 'Unifamiliars aïllats', 
            2: 'Unifamiliars adossats', 
            3: 'Plurifamiliars en bloc obert', 
            4: 'Plurifamiliars en bloc tancat'}

    mapping1 = {1: "De nova Construcció",
                2: "Rehabilitació integral"}

    mapping2 = {1: "Pendent d'enderroc", 
            2: "Solar", 
            3: "Buidat", 
            4: "Cimentació",
            5: "Estructura",
            6: "Tancaments exteriors",
            7: "Tancaments interiors",
            8: "Claus en mà",
            9: "NS/NC"}

    mapping3 = {
                    1: 'A',
                    2: 'B',
                    3: 'C',
                    4: 'D',
                    5: 'E',
                    6: "F",
                    7: "G",
                    8: "En tràmits",
                    9: "No info"
    }

    mapping4 = {
                    0: "Sense plaça d'aparcament",
                    1: "Plaça d'aparcament opcional",
                    2: "Plaça d'aparcament inclosa",
                    3: "Sense plaça d'aparcament",
    }


    # bbdd_estudi_hab['QENERGC'] = bbdd_estudi_hab['QENERGC'].map(number_to_letter_map)

    bbdd_estudi_prom['TIPO'] = bbdd_estudi_prom['TIPO'].map(mapping)

    bbdd_estudi_prom['TIPH'] = bbdd_estudi_prom['TIPH'].map(mapping1)


    bbdd_estudi_prom['ESTO'] = bbdd_estudi_prom['ESTO'].map(mapping2)

    bbdd_estudi_prom['QENERGC'] = bbdd_estudi_prom['QENERGC'].map(mapping3)

    bbdd_estudi_prom['APAR'] = bbdd_estudi_prom['APAR'].map(mapping4)


    # Importar BBDD habitatges
    bbdd_estudi_hab = pd.read_excel(path + 'BBDD 2022_2021 03.02.23.xlsx', sheet_name='Habitatges 2022_2021')
    bbdd_estudi_hab.columns = bbdd_estudi_hab.iloc[0,:]
    bbdd_estudi_hab = bbdd_estudi_hab[bbdd_estudi_hab["ESTUDI"]==any]





    # ["Total dormitoris","Banys i lavabos","Cuines estàndard","Cuines americanes","Terrasses, balcons i patis","Estudi/golfes","Safareig","Altres interiors","Altres exteriors"]

    # ["DORM", "LAV", "cuina_normal", "cuina_amer", "TER", "Golfes", "Safareig","Altres interiors","Altres exteriors" ]

    bbdd_estudi_hab['TIPOG'] = np.where(bbdd_estudi_hab['TIPO'].isin([1,2]), 'Habitatges Unifamiliars', 'Habitatges Plurifamiliars')
    bbdd_estudi_hab['TIPO'] = bbdd_estudi_hab['TIPO'].map(mapping)
    bbdd_estudi_hab['QENERGC'] = bbdd_estudi_hab['QENERGC'].map(mapping3)
    bbdd_estudi_hab['APAR'] = bbdd_estudi_hab['APAR'].map(mapping4)

    bbdd_estudi_hab = bbdd_estudi_hab.dropna(axis=1 , how ='all')



    bbdd_estudi_hab = bbdd_estudi_hab.rename(columns = {'V0006':'Total dormitoris_aux', 
                                                            "DORM": "Total dormitoris",
                                                            "LAV": "Banys i lavabos",
                                                            "TER": "Terrasses, balcons i patis",
                                                            'NOMD01C':'Superfície útil',
                                                            "Preu_m2_util": "Preu m2 útil",
                                                            "NOMD01F_2022": "Preu mitjà",
                                                            'NOMD01P':'Estudi/golfes', 
                                                            'NOMD01Q':'Safareig', 
                                                            'NOMD01K': 'Cuines estàndard', 
                                                            'NOMD01L': 'Cuines americanes', 
                                                            "NOMD01R": "Altres interiors", 
                                                            "NOMD01S":"Altres exteriors"})

    bbdd_estudi_prom = bbdd_estudi_prom.rename(columns = {'V0006':'Total dormitoris_aux', 
                                                            "DORM": "Total dormitoris",
                                                            "LAV": "Banys i lavabos",
                                                            "TER": "Terrasses, balcons i patis",
                                                            'NOMD01C':'Superfície útil',
                                                            "Preu_m2_util": "Preu m2 útil",
                                                            'NOMD01P':'Estudi/golfes', 
                                                            'NOMD01Q':'Safareig', 
                                                            'NOMD01K': 'Cuines estàndard', 
                                                            'NOMD01L': 'Cuines americanes', 
                                                            "NOMD01R": "Altres interiors", 
                                                            "NOMD01S":"Altres exteriors"})


    # Canviar de nom tots els equipaments
    bbdd_estudi_hab = bbdd_estudi_hab.rename(columns = {'EQUIC_1': 'Zona enjardinada', 
                                                        'EQUIC_2': 'Parc infantil',
                                                        'EQUIC_3': 'Piscina comunitària', 
                                                        'EQUIC_4': 'Traster', 
                                                        'EQUIC_5': 'Ascensor', 
                                                        'EQUIC_6': 'Equipament Esportiu',  
                                                        'EQUIC_7': 'Sala de jocs', 
                                                        'EQUIC_8': 'Sauna', 
                                                        "EQUIC_9_50": "Altres",
                                                        'EQUIC_99': 'Cap dels anteriors'})
    bbdd_estudi_prom = bbdd_estudi_prom.rename(columns = {'EQUIC_1': 'Zona enjardinada', 
                                                        'EQUIC_2': 'Parc infantil',
                                                        'EQUIC_3': 'Piscina comunitària', 
                                                        'EQUIC_4': 'Traster', 
                                                        'EQUIC_5': 'Ascensor', 
                                                        'EQUIC_6': 'Equipament Esportiu',  
                                                        'EQUIC_7': 'Sala de jocs', 
                                                        'EQUIC_8': 'Sauna', 
                                                        "QUAL_ALTRES": "Altres",
                                                        'EQUIC_99': 'Cap dels anteriors'})



    # Canviar de nom totes les qualitats
    bbdd_estudi_hab = bbdd_estudi_hab.rename(columns = {"QUALIC_5": "Aire condicionat", 
                                                        "QUALIC_6": "Bomba de calor", 
                                                        "QUALI_A": "Aero", 
                                                        'QUALIC_7':"Calefacció", 
                                                        'QUALIC_8':"Preinstal·lació d'A.C./B. Calor/Calefacció", 
                                                        'QUALIC_9': 'Parquet', 
                                                        'QUALIC_10':'Armaris encastats',
                                                        'QUALIC_12':'Placa de cocció amb gas',
                                                        'QUALIC_13':'Placa de cocció vitroceràmica',
                                                        "QUALIC_14":"Placa d'inducció",
                                                        'QUALIC_22':'Plaques solars'})


    bbdd_estudi_prom = bbdd_estudi_prom.rename(columns = {"QUALIC_5": "Aire condicionat", 
                                                        "QUALIC_6": "Bomba de calor", 
                                                        "QUALI_A": "Aero", 
                                                        'QUALIC_7':"Calefacció", 
                                                        'QUALIC_8':"Preinstal·lació d'A.C./B. Calor/Calefacció", 
                                                        'QUALIC_9': 'Parquet', 
                                                        'QUALIC_10':'Armaris encastats',
                                                        'QUALIC_12':'Placa de cocció amb gas',
                                                        'QUALIC_13':'Placa de cocció vitroceràmica',
                                                        "QUALIC_14":"Placa d'inducció",
                                                        'QUALIC_22':'Plaques solars'})
    #  Canviar nom a tipus de calefacció
    bbdd_estudi_prom = bbdd_estudi_prom.rename(columns = {'CALEFC_3': 'De gasoil', 
                                                        'CALEFC_4': 'De gas natural', 
                                                        'CALEFC_5': 'De propà', 
                                                        'CALEFC_6': "D'electricitat", 
                                                        'CALEFC_9': "No s'indica tipus"})




    bbdd_estudi_prom['TIPV'] = np.where(bbdd_estudi_prom['TIPV_1'] >= 1, "Venda a través d'immobiliària independent",
                                        np.where(bbdd_estudi_prom['TIPV_2'] >= 1, "Venda a través d'immobiliaria del mateix promotor",
                                                np.where(bbdd_estudi_prom['TIPV_3'] >= 1, "Venda directa del promotor", "Sense informació")))


    bbdd_estudi_prom['TIPOL_VENDA'] = np.where(bbdd_estudi_prom['TIPOL_VENDA_1'] == 1, "0D",
                                        np.where(bbdd_estudi_prom['TIPOL_VENDA_2'] == 1, "1D",
                                                np.where(bbdd_estudi_prom['TIPOL_VENDA_3'] == 1, "2D",
                                                        np.where(bbdd_estudi_prom['TIPOL_VENDA_4'] == 1, "3D",
                                                            np.where(bbdd_estudi_prom['TIPOL_VENDA_5'] == 1, "4D", 
                                                                np.where(bbdd_estudi_prom['TIPOL_VENDA_6'] == 1, "5+D", "NA"))))))

                        
                                                    
    #  "Venda a través d'immobiliària independent", "Venda a través d'immobiliaria del mateix promotor", "Venda directa del promotor"

#     bbdd_estudi_hab['TIPH'] = bbdd_estudi_hab['TIPH'].map(mapping1)

#     bbdd_estudi_hab['ESTO'] = bbdd_estudi_hab['ESTO'].map(mapping2)


#     vars = ['Zona enjardinada', 'Parc infantil', 'Piscina comunitària', 
#             'Traster', 'Ascensor', 'Equipament Esportiu', 'Sala de jocs', 
#             'Sauna', 'Altres', "Aire condicionat", "Bomba de calor", 
#             "Aero", "Calefacció", "Preinstal·lació d'A.C./B. Calor/Calefacció", 
#             "Parquet", "Armaris encastats", 'Placa de cocció amb gas', 
#             'Placa de cocció vitroceràmica', "Placa d'inducció", 'Plaques solars', "APAR"]
#     vars_aux = ['Zona enjardinada', 'Parc infantil', 'Piscina comunitària', 
#             'Traster', 'Ascensor', 'Equipament Esportiu', 'Sala de jocs', 
#             'Sauna', 'Altres', "Aire condicionat", "Bomba de calor", 
#             "Aero", "Calefacció", "Preinstal·lació d'A.C./B. Calor/Calefacció", 
#             "Parquet", "Armaris encastats", 'Placa de cocció amb gas', 
#             'Placa de cocció vitroceràmica', "Placa d'inducció", 'Plaques solars', "Safareig","Terrasses, balcons i patis"]
#     for i in vars:
#         bbdd_estudi_prom[i] = bbdd_estudi_prom[i].replace(np.nan, 0)
#     for i in vars_aux:
#         bbdd_estudi_hab[i] = bbdd_estudi_hab[i].replace(np.nan, 0)
#     bbdd_estudi_hab["Calefacció"] = bbdd_estudi_hab["Calefacció"].replace(' ', 0) 
#     bbdd_estudi_prom["Calefacció"] = bbdd_estudi_prom["Calefacció"].replace(' ', 0) 


    bbdd_estudi_hab["Tram_Sup_util"] = bbdd_estudi_hab["Tram_Sup_util"].str.replace(" ", "")
    bbdd_estudi_hab["Tram_Sup_util"] = bbdd_estudi_hab["Tram_Sup_util"].str[3:]



    # Afegir categories a algunes columnes de la base de dades d'habitatge

    room_dict =  {i: f"{i}D" if i <= 4 else "5+D" for i in range(0, 20)}
    toilet_dict = {i: f"{i} Bany" if i <= 1 else "2 i més Banys" for i in range(1, 20)}
    bbdd_estudi_hab_mod = bbdd_estudi_hab.copy()

    bbdd_estudi_hab_mod['Total dormitoris'] = bbdd_estudi_hab_mod['Total dormitoris'].map(room_dict)
    bbdd_estudi_hab_mod['Banys i lavabos'] = bbdd_estudi_hab_mod['Banys i lavabos'].map(toilet_dict)
    bbdd_estudi_hab_mod["Terrasses, balcons i patis"] = np.where(bbdd_estudi_hab_mod["Terrasses, balcons i patis"]>=1, 1, 0)

    return([bbdd_estudi_prom, bbdd_estudi_hab, bbdd_estudi_hab_mod])


bbdd_estudi_prom_2021, bbdd_estudi_hab_2021, bbdd_estudi_hab_mod_2021 = tidy_bbdd(2022)

In [5]:
def order_columns(table_n):
    cols = ['País', 'Província', 'Municipi'] + [col for col in table_n.columns if col not in ['País', 'Província', 'Municipi']]
    table_n = table_n.reindex(columns=cols)
    return(table_n)

PESTAÑA 9 - 14

In [6]:


table11_prov = bbdd_estudi_prom[["PROVINCIA", "HABIP", "HABIT"]].groupby("PROVINCIA").sum().reset_index()
table11_mun = bbdd_estudi_prom[["PROVINCIA","Municipi", "HABIP", "HABIT"]].groupby(["PROVINCIA","Municipi"]).sum().reset_index()

table11_hab = table11_mun.append(table11_prov, ignore_index=True)
table11_hab["País"] = "Catalunya"

table11_hab = table11_hab.append({"País":"Catalunya", "PROVINCIA": np.NaN, "Municipi":np.NaN, "HABIP":bbdd_estudi_prom["HABIP"].sum(), "HABIT":bbdd_estudi_prom["HABIT"].sum()}, ignore_index=True)
table11_hab = table11_hab.rename(columns={"HABIP":"Habitatges en oferta", "HABIT":"Habitatges a la promoció","PROVINCIA":"Província"})
table11_hab = table11_hab[["País", "Província", "Municipi", "Habitatges en oferta", "Habitatges a la promoció"]]


table11prom_prov = bbdd_estudi_prom[["PROVINCIA"]].value_counts().reset_index().rename(columns={0:"Promocions"})
table11prom_prov["Municipi"] = np.NaN
table11prom_mun = bbdd_estudi_prom[["PROVINCIA", "Municipi"]].value_counts().reset_index().rename(columns = {0:"Promocions"})
table11prom = pd.concat([table11prom_prov,table11prom_mun], axis=0)
table11prom["País"] = "Catalunya"

table11prom = table11prom.append({"País":"Catalunya", "PROVINCIA": np.NaN, "Municipi":np.NaN, "Promocions":bbdd_estudi_prom.shape[0]}, ignore_index=True)
table11prom = table11prom.rename(columns={"PROVINCIA":"Província"}).sort_values(["País", "Província", "Municipi"], ascending= True)
table11 = pd.merge(table11prom, table11_hab, how="left", on =["País", "Província", "Municipi"])
table11 = table11[["País", "Província", "Municipi", "Promocions", "Habitatges en oferta", "Habitatges a la promoció"]]

table11["Mitjana (en oferta)"] = round(table11["Habitatges en oferta"]/table11["Promocions"], 1)
table11["Mitjana (a la promoció)"] = round(table11["Habitatges a la promoció"]/table11["Promocions"], 1)
table11["% habitatges en venda"] = round(table11["Habitatges en oferta"].mul(100)/table11["Habitatges a la promoció"], 1)
table11 = table11[["País", "Província", "Municipi", "Promocions", "Habitatges en oferta", "Mitjana (en oferta)", "Habitatges a la promoció", "Mitjana (a la promoció)", "% habitatges en venda"]].sort_values(["Província", "Municipi"], na_position="first")
table11


,País,Província,Municipi,Promocions,Habitatges en oferta,Mitjana (en oferta),Habitatges a la promoció,Mitjana (a la promoció),% habitatges en venda
88,Catalunya,NaN,NaN,928,8181,8.8,21796,23.5,37.5
46,Catalunya,Barcelona,NaN,730,6190,8.5,17121,23.5,36.2
0,Catalunya,Barcelona,Arenys de Mar,8,76,9.5,229,28.6,33.2
1,Catalunya,Barcelona,Badalona,42,258,6.1,1040,24.8,24.8
2,Catalunya,Barcelona,Barcelona,199,1390,7.0,3278,16.5,42.4
...,...,...,...,...,...,...,...,...,...
82,Catalunya,Tarragona,Torredembarra,2,8,4.0,28,14.0,28.6
83,Catalunya,Tarragona,Tortosa,5,19,3.8,188,37.6,10.1
84,Catalunya,Tarragona,Valls,1,6,6.0,10,10.0,60.0
85,Catalunya,Tarragona,"Vendrell, el",4,51,12.8,68,17.0,75.0


PESTANYA 15 - 18

In [7]:
table17_hab_prov = bbdd_estudi_hab[["PROVINCIA", "ESTO"]].value_counts().reset_index().sort_values(["PROVINCIA", "ESTO"])
table17_hab_prov.columns = ["PROVINCIA","ESTAT", "PROMOCIONS"]
table17_hab_prov = table17_hab_prov.pivot_table(index="PROVINCIA", columns="ESTAT", values="PROMOCIONS").reset_index()
table17_hab_prov = table17_hab_prov[["PROVINCIA","Claus en mà"]].rename(columns={"PROVINCIA": "Província","Claus en mà":"Acabats sobre habitatges en oferta"})
table17_hab_prov["Municipi"] = np.NaN
table17_hab_prov["País"] = np.NaN

table17_hab_mun = bbdd_estudi_hab[["PROVINCIA", "Municipi", "ESTO"]].value_counts().reset_index().sort_values(["PROVINCIA", "ESTO"])
table17_hab_mun.columns = ["PROVINCIA", "Municipi","ESTAT", "PROMOCIONS"]
table17_hab_mun = table17_hab_mun.pivot_table(index=["PROVINCIA", "Municipi"], columns="ESTAT", values="PROMOCIONS").reset_index()
table17_hab_mun = table17_hab_mun[["PROVINCIA", "Municipi","Claus en mà"]]
table17_hab_mun["País"] = np.NaN
table17_hab_mun = table17_hab_mun[["País", "PROVINCIA", "Municipi", "Claus en mà"]].rename(columns={"PROVINCIA": "Província","Claus en mà":"Acabats sobre habitatges en oferta"})
table17_aux = pd.concat([table17_hab_prov, table17_hab_mun], axis=0)
table17_aux["País"] = "Catalunya"
table17_aux = table17_aux.append({"País":"Catalunya", "Província": np.NaN, "Municipi":np.NaN, "Acabats sobre habitatges en oferta":table17_aux[~table17_aux["Municipi"].isna()]["Acabats sobre habitatges en oferta"].sum()}, ignore_index=True)

table17_aux1 = pd.merge(table17_aux, table11[["País", "Província", "Municipi", "Habitatges en oferta"]], how="left", on=["País", "Província", "Municipi"])
table17_aux1 = table17_aux1[["País", "Província", "Municipi", "Habitatges en oferta", "Acabats sobre habitatges en oferta"]]
table17_aux1["En construcció sobre habitatges en oferta"] = table17_aux1["Habitatges en oferta"] - table17_aux1["Acabats sobre habitatges en oferta"]


table17_totalfinprov = pd.merge(bbdd_estudi_prom[["PROVINCIA","HABIT"]].groupby(["PROVINCIA"]).sum().reset_index().rename(columns= {"HABIT":"Total habitatges a la promoció"})
                                , bbdd_estudi_prom[bbdd_estudi_prom["ESTO"]=="Claus en mà"][["PROVINCIA","HABIT"]].groupby(["PROVINCIA"]).sum().reset_index().rename(columns={"HABIT": "Acabats sobre el total d'habitatges"}), how="left", on="PROVINCIA")
table17_totalfinprov["Municipi"] = np.NaN
table17_totalfinprov["País"] = "Catalunya"

table17_totalfinmun = pd.merge(bbdd_estudi_prom[["PROVINCIA","Municipi","HABIT"]].groupby(["PROVINCIA", "Municipi"]).sum().reset_index().rename(columns= {"HABIT":"Total habitatges a la promoció"})
                                , bbdd_estudi_prom[bbdd_estudi_prom["ESTO"]=="Claus en mà"][["PROVINCIA", "Municipi","HABIT"]].groupby(["PROVINCIA", "Municipi"]).sum().reset_index().rename(columns={"HABIT": "Acabats sobre el total d'habitatges"}), how="left", on=["PROVINCIA", "Municipi"])
table17_totalfinmun["País"] = "Catalunya"
table17_totaldict = table17_totalfinmun.set_index(["PROVINCIA", "Municipi"]).sum(axis=0).to_dict()
table17_totaldict.update({"País":"Catalunya"})
table17_totalfinmun = table17_totalfinmun.append(table17_totaldict, ignore_index=True)

table17_total = pd.concat([table17_totalfinprov, table17_totalfinmun], axis=0).rename(columns = {"PROVINCIA":"Província"})

table17 = pd.merge(table17_aux1, table17_total, how="left", on=["País","Província", "Municipi"])
table17["En construcció sobre el total d'habitatges"] = table17["Total habitatges a la promoció"] - table17["Acabats sobre el total d'habitatges"]

table17["% total sobre habitatges en oferta"] = round((table17["Habitatges en oferta"]/table17["Total habitatges a la promoció"])*100, 1)
table17["% acabats sobre habitatges en oferta"] = round((table17["Acabats sobre habitatges en oferta"]/table17["Acabats sobre el total d'habitatges"])*100, 1)
table17["% en construcció sobre habitatges en oferta"] = round((table17["En construcció sobre habitatges en oferta"]/table17["En construcció sobre el total d'habitatges"])*100, 1)
table17 = table17.sort_values(["Província", "Municipi"], na_position="first")
table17
# bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Calafell"][["PROVINCIA", "Municipi", "ESTO","HABIT"]].groupby(["PROVINCIA", "Municipi", "ESTO"]).sum()
# Atenció! Habitatges de Figueres contabilitzats 18, però a l'estudi d'ferta són 39. Verificat!
# table17_hab_mun[["Província","Acabats"]].groupby(["Província"]).sum().reset_index()
# table17_hab_mun[table17_hab_mun["Província"]=="Girona"]

,País,Província,Municipi,Habitatges en oferta,Acabats sobre habitatges en oferta,En construcció sobre habitatges en oferta,Total habitatges a la promoció,Acabats sobre el total d'habitatges,En construcció sobre el total d'habitatges,% total sobre habitatges en oferta,% acabats sobre habitatges en oferta,% en construcció sobre habitatges en oferta
88,Catalunya,NaN,NaN,8181,2461.0,5720.0,21796,9730.0,12066.0,37.5,25.3,47.4
0,Catalunya,Barcelona,NaN,6190,1618.0,4572.0,17121,6879.0,10242.0,36.2,23.5,44.6
4,Catalunya,Barcelona,Arenys de Mar,76,9.0,67.0,229,22.0,207.0,33.2,40.9,32.4
5,Catalunya,Barcelona,Badalona,258,33.0,225.0,1040,263.0,777.0,24.8,12.5,29.0
6,Catalunya,Barcelona,Barcelona,1390,577.0,813.0,3278,1433.0,1845.0,42.4,40.3,44.1
...,...,...,...,...,...,...,...,...,...,...,...,...
83,Catalunya,Tarragona,Torredembarra,8,8.0,0.0,28,28.0,0.0,28.6,28.6,NaN
84,Catalunya,Tarragona,Tortosa,19,19.0,0.0,188,188.0,0.0,10.1,10.1,NaN
85,Catalunya,Tarragona,Valls,6,6.0,0.0,10,10.0,0.0,60.0,60.0,NaN
86,Catalunya,Tarragona,"Vendrell, el",51,41.0,10.0,68,57.0,11.0,75.0,71.9,90.9


PESTANYA 19 - 22

In [8]:

table33_prov =  pd.crosstab(bbdd_estudi_hab_mod["PROVINCIA"], bbdd_estudi_hab_mod["Total dormitoris"]).reset_index().rename(columns={"PROVINCIA":"Província"})
table33_prov["Municipi"] = np.NaN
table33_prov["País"] = "Catalunya"

table33_mun = pd.crosstab(bbdd_estudi_hab_mod["Municipi"], bbdd_estudi_hab_mod["Total dormitoris"]).reset_index()
table33_mun = pd.merge(table33_mun, table11[["Província", "Municipi"]], how ="left", on="Municipi")
table33_mun["País"] = "Catalunya"


table33_dict = bbdd_estudi_hab_mod["Total dormitoris"].value_counts().to_dict()
table33_dict.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN})


table33 = pd.concat([table33_prov, table33_mun], axis=0)
table33 = table33.append(table33_dict, ignore_index=True)

table33 = pd.merge(table11[["País", "Província", "Municipi", "Habitatges en oferta"]], table33, how="left", on=["País","Província", "Municipi"])

cols = ['País', 'Província', 'Municipi'] + [col for col in table33.columns if col not in ['País', 'Província', 'Municipi']]
table33 = table33.reindex(columns=cols)

for i in ["0D", "1D", "2D", "3D", "4D", "5+D"]:
    table33[i +" %"] = round(table33[i].mul(100)/table33["Habitatges en oferta"],1)
table33 = table33.sort_values(["Província", "Municipi"], na_position="first")
table33


,País,Província,Municipi,Habitatges en oferta,0D,1D,2D,3D,4D,5+D,0D %,1D %,2D %,3D %,4D %,5+D %
0,Catalunya,NaN,NaN,8181,39,431,2432,4138,1088,53,0.5,5.3,29.7,50.6,13.3,0.6
1,Catalunya,Barcelona,NaN,6190,39,374,1836,3132,772,37,0.6,6.0,29.7,50.6,12.5,0.6
2,Catalunya,Barcelona,Arenys de Mar,76,0,0,22,49,5,0,0.0,0.0,28.9,64.5,6.6,0.0
3,Catalunya,Barcelona,Badalona,258,0,27,57,168,6,0,0.0,10.5,22.1,65.1,2.3,0.0
4,Catalunya,Barcelona,Barcelona,1390,25,167,579,493,113,13,1.8,12.0,41.7,35.5,8.1,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Catalunya,Tarragona,Torredembarra,8,0,1,5,2,0,0,0.0,12.5,62.5,25.0,0.0,0.0
85,Catalunya,Tarragona,Tortosa,19,0,0,8,10,1,0,0.0,0.0,42.1,52.6,5.3,0.0
86,Catalunya,Tarragona,Valls,6,0,6,0,0,0,0,0.0,100.0,0.0,0.0,0.0,0.0
87,Catalunya,Tarragona,"Vendrell, el",51,0,0,20,13,18,0,0.0,0.0,39.2,25.5,35.3,0.0


PESTANYA 23 - 30

In [9]:
# Table4: Total d'habitatges en oferta segons Nova construcció o rehabilitació
table38prom_prov = bbdd_estudi_prom[["PROVINCIA", "TIPH"]].value_counts().reset_index().sort_values(["PROVINCIA", "TIPH"])
table38prom_prov.columns = ["PROVINCIA", "TIPOLOGIA", "PROMOCIONS"]
table38prom_prov = table38prom_prov.pivot_table(index="PROVINCIA", columns="TIPOLOGIA", values="PROMOCIONS").reset_index().rename(columns={"PROVINCIA":"Província"})
table38prom_prov["País"] = "Catalunya"

table38prom_mun = bbdd_estudi_prom[["Municipi", "TIPH"]].value_counts().reset_index().sort_values(["Municipi", "TIPH"])
table38prom_mun.columns = ["Municipi", "TIPOLOGIA", "PROMOCIONS"]
table38prom_mun = table38prom_mun.pivot_table(index="Municipi", columns="TIPOLOGIA", values="PROMOCIONS").reset_index()
table38prom_mun = pd.merge(table11[["Província", "Municipi"]].dropna(), table38prom_mun, how="left", on = "Municipi")
table38prom_mun["País"] = "Catalunya"
table38prom_dict = table38prom_mun.set_index(["Província","Municipi"]).sum(axis=0).to_dict()
table38prom_dict.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN})
table38prom_mun = table38prom_mun.append(table38prom_dict, ignore_index=True)

table38hab_prov = bbdd_estudi_hab[["PROVINCIA", "TIPH"]].value_counts().reset_index().sort_values(["PROVINCIA", "TIPH"])
table38hab_prov.columns = ["PROVINCIA", "TIPOLOGIA", "Habitatges"]
table38hab_prov = table38hab_prov.pivot_table(index="PROVINCIA", columns="TIPOLOGIA", values="Habitatges").reset_index().rename(columns={"PROVINCIA":"Província"})
table38hab_prov["País"] = "Catalunya"

table38hab_mun = bbdd_estudi_hab[["Municipi", "TIPH"]].value_counts().reset_index().sort_values(["Municipi", "TIPH"])
table38hab_mun.columns = ["Municipi", "TIPOLOGIA", "Habitatges"]
table38hab_mun = table38hab_mun.pivot_table(index="Municipi", columns="TIPOLOGIA", values="Habitatges").reset_index()
table38hab_mun = pd.merge(table11[["Província", "Municipi"]].dropna(), table38hab_mun, how="left", on = "Municipi")
table38hab_mun["País"] = "Catalunya"
table38hab_dict = table38hab_mun.set_index(["Província","Municipi"]).sum(axis=0).to_dict()
table38hab_dict.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN})
table38hab_mun = table38hab_mun.append(table38hab_dict, ignore_index=True)


table38_prom = pd.concat([table38prom_prov, table38prom_mun], axis=0)
table38_prom = pd.merge(table11[["País", "Província", "Municipi", "Promocions"]], table38_prom, how="left", on=["País", "Província", "Municipi"])
table38_prom = order_columns(table38_prom)


table38_hab = pd.concat([table38hab_prov, table38hab_mun], axis=0)
table38_hab = pd.merge(table11[["País", "Província", "Municipi", "Habitatges en oferta"]], table38_hab, how="left", on=["País", "Província", "Municipi"])
table38_hab = order_columns(table38_hab)

for i in ["De nova Construcció","Rehabilitació integral"]:
    table38_hab[i + " %"] = round(table38_hab[i].mul(100)/table38_hab["Habitatges en oferta"], 1)
    table38_prom[i + " %"] = round(table38_prom[i].mul(100)/table38_prom["Promocions"],1)


table38 = table38_prom.sort_values(["Província", "Municipi"], na_position="first").copy()
table42 = table38_hab.sort_values(["Província", "Municipi"], na_position="first").copy()
table38

,País,Província,Municipi,Promocions,De nova Construcció,Rehabilitació integral,De nova Construcció %,Rehabilitació integral %
0,Catalunya,NaN,NaN,928,853.0,75.0,91.9,8.1
1,Catalunya,Barcelona,NaN,730,662.0,68.0,90.7,9.3
2,Catalunya,Barcelona,Arenys de Mar,8,8.0,NaN,100.0,NaN
3,Catalunya,Barcelona,Badalona,42,42.0,NaN,100.0,NaN
4,Catalunya,Barcelona,Barcelona,199,138.0,61.0,69.3,30.7
...,...,...,...,...,...,...,...,...
84,Catalunya,Tarragona,Torredembarra,2,2.0,NaN,100.0,NaN
85,Catalunya,Tarragona,Tortosa,5,5.0,NaN,100.0,NaN
86,Catalunya,Tarragona,Valls,1,1.0,NaN,100.0,NaN
87,Catalunya,Tarragona,"Vendrell, el",4,4.0,NaN,100.0,NaN


PESTANYA 31

In [10]:
bbdd_estudi_hab_mod["TIPOG_aux"] = "Total habitatges"
table76_tipo = bbdd_estudi_hab_mod[["Total dormitoris", "TIPOG","Superfície útil", "Preu mitjà", "Preu m2 útil"]].set_index(["Total dormitoris","TIPOG"]).groupby(["TIPOG", "Total dormitoris"]).apply(lambda x: pd.Series({
    "Superfície útil": np.mean(x["Superfície útil"]),
    "Preu mitjà": np.mean(x["Preu mitjà"]),
    "Preu m2 útil": np.mean(x["Preu m2 útil"]),
    "Unitats": len(x)
})).reset_index()
table76_tipo
table76_total = bbdd_estudi_hab_mod[["TIPOG_aux","Total dormitoris","Superfície útil", "Preu mitjà", "Preu m2 útil"]].set_index(["TIPOG_aux", "Total dormitoris"]).groupby(["TIPOG_aux", "Total dormitoris"]).apply(lambda x: pd.Series({
    "Superfície útil": np.mean(x["Superfície útil"]),
    "Preu mitjà": np.mean(x["Preu mitjà"]),
    "Preu m2 útil": np.mean(x["Preu m2 útil"]),
    "Unitats": len(x)
})).reset_index().drop("TIPOG_aux", axis=1)

table76_aux1 = bbdd_estudi_hab_mod[["TIPOG_aux", "Total dormitoris","Superfície útil", "Preu mitjà", "Preu m2 útil"]].set_index(["TIPOG_aux"]).groupby(["TIPOG_aux"]).apply(lambda x: pd.Series({
    "Superfície útil": np.mean(x["Superfície útil"]),
    "Preu mitjà": np.mean(x["Preu mitjà"]),
    "Preu m2 útil": np.mean(x["Preu m2 útil"]),
    "Unitats": len(x)
})).reset_index().drop("TIPOG_aux", axis=1)
table76_aux1["Total dormitoris"] = "Total"
table76_aux1["TIPOG"] = "Total habitatges"

table76_aux2 = bbdd_estudi_hab_mod[["TIPOG","Total dormitoris","Superfície útil", "Preu mitjà", "Preu m2 útil"]].set_index(["TIPOG", "Total dormitoris"]).groupby(["TIPOG"]).apply(lambda x: pd.Series({
    "Superfície útil": np.mean(x["Superfície útil"]),
    "Preu mitjà": np.mean(x["Preu mitjà"]),
    "Preu m2 útil": np.mean(x["Preu m2 útil"]),
    "Unitats": len(x)
})).reset_index()
table76_aux2["Total dormitoris"] = "Total"
table76_aux2["TIPOG"]

table76_total["TIPOG"] = "Total habitatges"
table76 = pd.concat([table76_tipo, table76_total, table76_aux1, table76_aux2], axis=0)
table76_melted = pd.melt(table76, id_vars=["TIPOG", "Total dormitoris"])
table76 = table76_melted.pivot(index="Total dormitoris", columns=["TIPOG", "variable"], values="value")
table76 = table76.sort_index(level=0, axis=1)
table76


TIPOG            Habitatges Plurifamiliars                                \
variable                      Preu m2 útil    Preu mitjà Superfície útil   
Total dormitoris                                                           
0D                             6406.150665  4.395096e+05       66.244314   
1D                             5151.715774  2.736605e+05       50.732121   
2D                             4841.591729  3.216417e+05       64.456211   
3D                             4373.149481  3.634942e+05       81.497028   
4D                             4419.581741  4.788672e+05      104.733043   
5+D                            6965.777765  1.176996e+06      160.499085   
Total                          4585.608817  3.624916e+05       77.481170   

TIPOG                    Habitatges Unifamiliars                \
variable         Unitats            Preu m2 útil    Preu mitjà   
Total dormitoris                                                 
0D                  39.0                     NaN           NaN   
1D                 431.0                     NaN           NaN   
2D                2425.0             3646.180371  4.095714e+05   
3D                3996.0             3219.185524  4.548167e+05   
4D                 924.0             3214.170432  4.711303e+05   
5+D                 28.0             4255.035036  1.179440e+06   
Total             7843.0             3302.211343  5.153916e+05   

TIPOG                                    Total habitatges                \
variable         Superfície útil Unitats     Preu m2 útil    Preu mitjà   
Total dormitoris                                                          
0D                           NaN     NaN      6406.150665  4.395096e+05   
1D                           NaN     NaN      5151.715774  2.736605e+05   
2D                    113.581429     7.0      4838.150989  3.218948e+05   
3D                    146.297972   142.0      4333.549945  3.666280e+05   
4D                    148.792119   164.0      4237.883713  4.777010e+05   
5+D                   263.724863    25.0      5687.125534  1.178149e+06   
Total                 155.516009   338.0      4532.584939  3.688087e+05   

TIPOG                                     
variable         Superfície útil Unitats  
Total dormitoris                          
0D                     66.244314    39.0  
1D                     50.732121   431.0  
2D                     64.597607  2432.0  
3D                     83.720743  4138.0  
4D                    111.374301  1088.0  
5+D                   209.190490    53.0  
Total                  80.705198  8181.0

PESTANYA 32 - 40

In [42]:

table80uni_prov = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["TIPOG"]=="Habitatges Unifamiliars"][["PROVINCIA", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA"]).agg({"PROVINCIA":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"PROVINCIA":"Unitats"}).reset_index()


table80uni_prov.columns = pd.MultiIndex.from_tuples([("Habitatges Unifamiliars", "Província"),
            # ("Habitatges Unifamiliars", "Municipi"),
            ("Habitatges Unifamiliars", "Unitats"),
           ("Habitatges Unifamiliars", "Superfície útil"),
           ("Habitatges Unifamiliars", "Preu mitjà"),
           ("Habitatges Unifamiliars", "Preu m2 útil")])
table80uni_prov[("Habitatges Unifamiliars","Municipi")] = np.NaN
table80uni_prov[("Habitatges Unifamiliars", "País")] = "Catalunya"

table80uni_mun = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["TIPOG"]=="Habitatges Unifamiliars"][["PROVINCIA","Municipi", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA", "Municipi"]).agg({"Municipi":['count'], "Superfície útil": [np.mean], 
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"PROVINCIA":"Unitats"}).reset_index()
table80uni_mun.columns = pd.MultiIndex.from_tuples([("Habitatges Unifamiliars", "Província"),
            ("Habitatges Unifamiliars", "Municipi"),
            ("Habitatges Unifamiliars", "Unitats"),
           ("Habitatges Unifamiliars", "Superfície útil"),
           ("Habitatges Unifamiliars", "Preu mitjà"),
           ("Habitatges Unifamiliars", "Preu m2 útil")])

table80uni_mun[("Habitatges Unifamiliars", "País")] = "Catalunya"

table80pluri_prov = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["TIPOG"]=="Habitatges Plurifamiliars"][["PROVINCIA", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA"]).agg({"PROVINCIA":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"PROVINCIA":"Unitats"}).reset_index()


table80pluri_prov.columns = pd.MultiIndex.from_tuples([("Habitatges Plurifamiliars", "Província"),
            # ("Habitatges Plurifamiliars", "Municipi"),
            ("Habitatges Plurifamiliars", "Unitats"),
           ("Habitatges Plurifamiliars", "Superfície útil"),
           ("Habitatges Plurifamiliars", "Preu mitjà"),
           ("Habitatges Plurifamiliars", "Preu m2 útil")])
table80pluri_prov[("Habitatges Plurifamiliars","Municipi")] = np.NaN
table80pluri_prov[("Habitatges Plurifamiliars", "País")] = "Catalunya"

table80pluri_mun = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["TIPOG"]=="Habitatges Plurifamiliars"][["PROVINCIA", "Municipi", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA", "Municipi"]).agg({"Municipi":['count'], 
                                                                                    "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], 
                                                                                    "Preu m2 útil": [np.mean]}).rename(columns={"PROVINCIA":"Unitats"}).reset_index()                                                                                   

table80pluri_mun.columns = pd.MultiIndex.from_tuples([("Habitatges Plurifamiliars", "Província"),
            ("Habitatges Plurifamiliars", "Municipi"),
            ("Habitatges Plurifamiliars", "Unitats"),
           ("Habitatges Plurifamiliars", "Superfície útil"),
           ("Habitatges Plurifamiliars", "Preu mitjà"),
           ("Habitatges Plurifamiliars", "Preu m2 útil")])
table80pluri_mun[("Habitatges Plurifamiliars", "País")] = "Catalunya"



table80_prov = bbdd_estudi_hab_mod[["PROVINCIA", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA"]).agg({"PROVINCIA":['count'],"Superfície útil": [np.mean], "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).reset_index()

table80_prov.columns = pd.MultiIndex.from_tuples([("Total habitatges", "Província"),
            # ("Total habitatges", "Municipi"),
            ("Total habitatges", "Unitats"),
           ("Total habitatges", "Superfície útil"),
           ("Total habitatges", "Preu mitjà"),
           ("Total habitatges", "Preu m2 útil")])

table80_prov[("Total habitatges", "Municipi")] = np.NaN
table80_prov[("Total habitatges", "País")] = "Catalunya"
    

table80_mun = bbdd_estudi_hab_mod[["PROVINCIA", "Municipi", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA","Municipi"]).agg({"Municipi":['count'], "Superfície útil": [np.mean], "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).reset_index()

table80_mun.columns = pd.MultiIndex.from_tuples([("Total habitatges", "Província"),
            ("Total habitatges", "Municipi"),
            ("Total habitatges", "Unitats"),
           ("Total habitatges", "Superfície útil"),
           ("Total habitatges", "Preu mitjà"),
           ("Total habitatges", "Preu m2 útil")])
table80_mun[("Total habitatges", "País")] = "Catalunya"




dict80_total = bbdd_estudi_hab_mod[["Superfície útil", "Preu mitjà", "Preu m2 útil"]].mean().to_dict()
dict80_total.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN, "Unitats":bbdd_estudi_hab_mod.shape[0]})

dict80_uni = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["TIPOG"]=="Habitatges Unifamiliars"][["Superfície útil", "Preu mitjà", "Preu m2 útil"]].mean().to_dict()
dict80_uni.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN, "Unitats":bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["TIPOG"]=="Habitatges Unifamiliars"].shape[0]})

dict80_pluri = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["TIPOG"]=="Habitatges Plurifamiliars"][["Superfície útil", "Preu mitjà", "Preu m2 útil"]].mean().to_dict()
dict80_pluri.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN, "Unitats":bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["TIPOG"]=="Habitatges Plurifamiliars"].shape[0]})

dict80_total_df = pd.DataFrame(dict80_total, index=[0])
dict80_total_df.columns = pd.MultiIndex.from_tuples([
           ("Total habitatges", "Superfície útil"),
           ("Total habitatges", "Preu mitjà"),
           ("Total habitatges", "Preu m2 útil"),
           ("Total habitatges", "País"),
           ("Total habitatges", "Província"),
            ("Total habitatges", "Municipi"),
            ("Total habitatges", "Unitats")])

dict80_uni_df = pd.DataFrame(dict80_uni, index=[0])
dict80_uni_df.columns = pd.MultiIndex.from_tuples([
           ("Habitatges Unifamiliars", "Superfície útil"),
           ("Habitatges Unifamiliars", "Preu mitjà"),
           ("Habitatges Unifamiliars", "Preu m2 útil"),
           ("Habitatges Unifamiliars", "País"),
           ("Habitatges Unifamiliars", "Província"),
            ("Habitatges Unifamiliars", "Municipi"),
            ("Habitatges Unifamiliars", "Unitats")])


dict80_pluri_df = pd.DataFrame(dict80_pluri, index=[0])
dict80_pluri_df.columns = pd.MultiIndex.from_tuples([
           ("Habitatges Plurifamiliars", "Superfície útil"),
           ("Habitatges Plurifamiliars", "Preu mitjà"),
           ("Habitatges Plurifamiliars", "Preu m2 útil"),
           ("Habitatges Plurifamiliars", "País"),
           ("Habitatges Plurifamiliars", "Província"),
            ("Habitatges Plurifamiliars", "Municipi"),
            ("Habitatges Plurifamiliars", "Unitats")])

table80_total = pd.concat([table80_prov, table80_mun, dict80_total_df], axis=0)
table80_pluri = pd.concat([table80pluri_prov, table80pluri_mun, dict80_pluri_df], axis=0)
table80_uni = pd.concat([table80uni_prov, table80uni_mun, dict80_uni_df], axis=0)
# table80_total = table80_total[[('Total habitatges',            'País'), ('Total habitatges',       'Província'),            ('Total habitatges',        'Municipi'),
#             ('Total habitatges',         'Unitats'),
#             ('Total habitatges', 'Superfície útil'),
#             ('Total habitatges',      'Preu mitjà'),
#             ('Total habitatges',    'Preu m2 útil')
#             ]]
# table80_pluri = table80_pluri[[('Habitatges Plurifamiliars',            'País'), ('Habitatges Plurifamiliars',       'Província'),            ('Habitatges Plurifamiliars',        'Municipi'),
#             ('Habitatges Plurifamiliars',         'Unitats'),
#             ('Habitatges Plurifamiliars', 'Superfície útil'),
#             ('Habitatges Plurifamiliars',      'Preu mitjà'),
#             ('Habitatges Plurifamiliars',    'Preu m2 útil')
#             ]]
# table80_uni= table80_uni[[('Habitatges Unifamiliars',            'País'), ('Habitatges Unifamiliars',       'Província'),            ('Habitatges Unifamiliars',        'Municipi'),
#             ('Habitatges Unifamiliars',         'Unitats'),
#             ('Habitatges Unifamiliars', 'Superfície útil'),
#             ('Habitatges Unifamiliars',      'Preu mitjà'),
#             ('Habitatges Unifamiliars',    'Preu m2 útil')
#             ]]

# table80_total = table80_total.sort_values([('Total habitatges',            'País'), ('Total habitatges',       'Província'),            ('Total habitatges',        'Municipi')], na_position="first")
# table80_pluri = table80_pluri.sort_values([('Habitatges Plurifamiliars',            'País'), ('Habitatges Plurifamiliars',       'Província'),            ('Habitatges Plurifamiliars',        'Municipi')], na_position="first")
# table80_uni = table80_uni.sort_values([('Habitatges Unifamiliars',            'País'), ('Habitatges Unifamiliars',       'Província'),            ('Habitatges Unifamiliars',        'Municipi')], na_position="first")

table80_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 0
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   (Total habitatges, Província)        88 non-null     object 
 1   (Total habitatges, Unitats)          89 non-null     int64  
 2   (Total habitatges, Superfície útil)  89 non-null     float64
 3   (Total habitatges, Preu mitjà)       89 non-null     float64
 4   (Total habitatges, Preu m2 útil)     89 non-null     float64
 5   (Total habitatges, Municipi)         84 non-null     object 
 6   (Total habitatges, País)             89 non-null     object 
dtypes: float64(3), int64(1), object(3)
memory usage: 5.6+ KB


In [41]:
table80_uni.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 0
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   (Habitatges Unifamiliars, Província)        49 non-null     object 
 1   (Habitatges Unifamiliars, Unitats)          50 non-null     int64  
 2   (Habitatges Unifamiliars, Superfície útil)  50 non-null     float64
 3   (Habitatges Unifamiliars, Preu mitjà)       50 non-null     float64
 4   (Habitatges Unifamiliars, Preu m2 útil)     50 non-null     float64
 5   (Habitatges Unifamiliars, Municipi)         45 non-null     object 
 6   (Habitatges Unifamiliars, País)             50 non-null     object 
dtypes: float64(3), int64(1), object(3)
memory usage: 3.1+ KB


In [13]:
table80_pluri

Habitatges Plurifamiliars                                    \
                        País  Província       Municipi Unitats   
0                  Catalunya        NaN            NaN    7843   
0                  Catalunya  Barcelona            NaN    6020   
0                  Catalunya  Barcelona  Arenys de Mar      70   
1                  Catalunya  Barcelona       Badalona     250   
2                  Catalunya  Barcelona      Barcelona    1380   
..                       ...        ...            ...     ...   
74                 Catalunya  Tarragona  Torredembarra       8   
75                 Catalunya  Tarragona        Tortosa      19   
76                 Catalunya  Tarragona          Valls       6   
77                 Catalunya  Tarragona   Vendrell, el      35   
78                 Catalunya  Tarragona      Vila-seca      22   

                                                
   Superfície útil     Preu mitjà Preu m2 útil  
0        77.481170  362491.583578  4585.608817  
0        77.308440  382726.480565  4824.044249  
0        75.917668  252178.000000  3456.843449  
1        77.827115  314912.188000  4095.516759  
2        80.053614  629114.031884  7509.952785  
..             ...            ...          ...  
74       59.132250  141787.500000  2471.340685  
75       68.070522  121161.894737  1745.531657  
76       35.833333   45166.666667  1263.924851  
77       75.579833  108542.857143  1429.910548  
78       82.367878  210181.818182  2552.653940  

[84 rows x 7 columns]

PESTANYA 41 - 44

In [14]:
table101_prov = bbdd_estudi_prom_2021[["PROVINCIA", "HABIP", "HABIT"]].groupby("PROVINCIA").sum().reset_index()
table101_mun = bbdd_estudi_prom_2021[["PROVINCIA","Municipi", "HABIP", "HABIT"]].groupby(["PROVINCIA","Municipi"]).sum().reset_index()

table101_hab = table101_mun.append(table101_prov, ignore_index=True)
table101_hab["País"] = "Catalunya"

table101_hab = table101_hab.append({"País":"Catalunya", "PROVINCIA": np.NaN, "Municipi":np.NaN, "HABIP":bbdd_estudi_prom_2021["HABIP"].sum(), "HABIT":bbdd_estudi_prom_2021["HABIT"].sum()}, ignore_index=True)
table101_hab = table101_hab.rename(columns={"HABIP":"Habitatges en oferta", "HABIT":"Habitatges a la promoció","PROVINCIA":"Província"})
table101_hab = table101_hab[["País", "Província", "Municipi", "Habitatges en oferta", "Habitatges a la promoció"]]


table101prom_prov = bbdd_estudi_prom_2021[["PROVINCIA"]].value_counts().reset_index().rename(columns={0:"Promocions"})
table101prom_prov["Municipi"] = np.NaN
table101prom_mun = bbdd_estudi_prom_2021[["PROVINCIA", "Municipi"]].value_counts().reset_index().rename(columns = {0:"Promocions"})
table101prom = pd.concat([table101prom_prov,table101prom_mun], axis=0)
table101prom["País"] = "Catalunya"

table101prom = table101prom.append({"País":"Catalunya", "PROVINCIA": np.NaN, "Municipi":np.NaN, "Promocions":bbdd_estudi_prom_2021.shape[0]}, ignore_index=True)
table101prom = table101prom.rename(columns={"PROVINCIA":"Província"}).sort_values(["País", "Província", "Municipi"], ascending= True)
table101 = pd.merge(table101prom, table101_hab, how="left", on =["País", "Província", "Municipi"])
table101 = table101[["País", "Província", "Municipi", "Promocions", "Habitatges en oferta", "Habitatges a la promoció"]]

table101["Mitjana (en oferta)"] = table101["Habitatges en oferta"]/table101["Promocions"]
table101["Mitjana (a la promoció)"] = table101["Habitatges a la promoció"]/table101["Promocions"]
table101["% habitatges en venda"] = table101["Habitatges en oferta"].mul(100)/table101["Habitatges a la promoció"]

table101_2021 = table101[["País", "Província", "Municipi", "Promocions", "Habitatges en oferta"]]
table101_2021.columns = ["País", "Província", "Municipi", "Promocions 2021", "Habitatges en oferta 2021"]
table101_2022 = table11[["País", "Província", "Municipi", "Promocions", "Habitatges en oferta"]]
table101_2022.columns = ["País", "Província", "Municipi", "Promocions 2022", "Habitatges en oferta 2022"]



table101_def = pd.merge(table101_2021, table101_2022, how="left", on=["País", "Província", "Municipi"])
table101_def["Variació Promocions"] = table101_def["Promocions 2022"] - table101_def["Promocions 2021"] 
table101_def["Variació Habitatges"] = table101_def["Habitatges en oferta 2022"] - table101_def["Habitatges en oferta 2021"] 


table101_def = table101_def.sort_values(["País", "Província", "Municipi"], na_position="first")
table101_def

,País,Província,Municipi,Promocions 2021,Habitatges en oferta 2021,Promocions 2022,Habitatges en oferta 2022,Variació Promocions,Variació Habitatges
98,Catalunya,NaN,NaN,1015,6294,928.0,8181.0,-87.0,1887.0
49,Catalunya,Barcelona,NaN,752,4878,730.0,6190.0,-22.0,1312.0
0,Catalunya,Barcelona,Arenys de Mar,7,52,8.0,76.0,1.0,24.0
1,Catalunya,Barcelona,Badalona,38,300,42.0,258.0,4.0,-42.0
2,Catalunya,Barcelona,Barcelona,237,1273,199.0,1390.0,-38.0,117.0
...,...,...,...,...,...,...,...,...,...
92,Catalunya,Tarragona,Torredembarra,4,15,2.0,8.0,-2.0,-7.0
93,Catalunya,Tarragona,Tortosa,5,53,5.0,19.0,0.0,-34.0
94,Catalunya,Tarragona,Valls,1,2,1.0,6.0,0.0,4.0
95,Catalunya,Tarragona,"Vendrell, el",4,36,4.0,51.0,0.0,15.0


PESTANYA 45

In [15]:
table116_tipo_tot = bbdd_estudi_hab_mod_2021[["Total dormitoris", "TIPOG","Superfície útil", "Preu mitjà", "Preu m2 útil"]].set_index(["Total dormitoris", "TIPOG"]).groupby(["TIPOG"]).apply(np.mean).reset_index()
table116_tipo_tot["Total dormitoris"] = "Total"
table116_tipo_tot.columns = ['TIPOG', 'Superfície útil 2021', 'Preu mitjà 2021', 'Preu m2 útil 2021',
       'Total dormitoris 2021']

table116_tipo_tot

,TIPOG,Superfície útil 2021,Preu mitjà 2021,Preu m2 útil 2021,Total dormitoris 2021
0,Habitatges Plurifamiliars,80.059431,368225.726480,4497.863309,Total
1,Habitatges Unifamiliars,170.905654,520009.728097,3048.250196,Total


PESTANYA 46 - 49

In [43]:
table117uni_prov = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["TIPOG"]=="Habitatges Unifamiliars"][["PROVINCIA", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA"]).agg({"PROVINCIA":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"PROVINCIA":"Unitats"}).reset_index()


table117uni_prov.columns = pd.MultiIndex.from_tuples([("Habitatges Unifamiliars", "Província"),
            # ("Habitatges Unifamiliars", "Municipi"),
            ("Habitatges Unifamiliars", "Unitats"),
           ("Habitatges Unifamiliars", "Superfície útil"),
           ("Habitatges Unifamiliars", "Preu mitjà"),
           ("Habitatges Unifamiliars", "Preu m2 útil")])
table117uni_prov[("Habitatges Unifamiliars","Municipi")] = np.NaN
table117uni_prov[("Habitatges Unifamiliars", "País")] = "Catalunya"

table117uni_mun = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["TIPOG"]=="Habitatges Unifamiliars"][["PROVINCIA","Municipi", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA", "Municipi"]).agg({"Municipi":['count'], "Superfície útil": [np.mean], 
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"PROVINCIA":"Unitats"}).reset_index()
table117uni_mun.columns = pd.MultiIndex.from_tuples([("Habitatges Unifamiliars", "Província"),
            ("Habitatges Unifamiliars", "Municipi"),
            ("Habitatges Unifamiliars", "Unitats"),
           ("Habitatges Unifamiliars", "Superfície útil"),
           ("Habitatges Unifamiliars", "Preu mitjà"),
           ("Habitatges Unifamiliars", "Preu m2 útil")])

table117uni_mun[("Habitatges Unifamiliars", "País")] = "Catalunya"

table117pluri_prov = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["TIPOG"]=="Habitatges Plurifamiliars"][["PROVINCIA", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA"]).agg({"PROVINCIA":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"PROVINCIA":"Unitats"}).reset_index()


table117pluri_prov.columns = pd.MultiIndex.from_tuples([("Habitatges Plurifamiliars", "Província"),
            # ("Habitatges Plurifamiliars", "Municipi"),
            ("Habitatges Plurifamiliars", "Unitats"),
           ("Habitatges Plurifamiliars", "Superfície útil"),
           ("Habitatges Plurifamiliars", "Preu mitjà"),
           ("Habitatges Plurifamiliars", "Preu m2 útil")])
table117pluri_prov[("Habitatges Plurifamiliars","Municipi")] = np.NaN
table117pluri_prov[("Habitatges Plurifamiliars", "País")] = "Catalunya"

table117pluri_mun = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["TIPOG"]=="Habitatges Plurifamiliars"][["PROVINCIA", "Municipi", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA", "Municipi"]).agg({"Municipi":['count'], 
                                                                                    "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], 
                                                                                    "Preu m2 útil": [np.mean]}).rename(columns={"PROVINCIA":"Unitats"}).reset_index()                                                                                   

table117pluri_mun.columns = pd.MultiIndex.from_tuples([("Habitatges Plurifamiliars", "Província"),
            ("Habitatges Plurifamiliars", "Municipi"),
            ("Habitatges Plurifamiliars", "Unitats"),
           ("Habitatges Plurifamiliars", "Superfície útil"),
           ("Habitatges Plurifamiliars", "Preu mitjà"),
           ("Habitatges Plurifamiliars", "Preu m2 útil")])
table117pluri_mun[("Habitatges Plurifamiliars", "País")] = "Catalunya"



table117_prov = bbdd_estudi_hab_mod_2021[["PROVINCIA", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA"]).agg({"PROVINCIA":['count'],"Superfície útil": [np.mean], "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).reset_index()

table117_prov.columns = pd.MultiIndex.from_tuples([("Total habitatges", "Província"),
            # ("Total habitatges", "Municipi"),
            ("Total habitatges", "Unitats"),
           ("Total habitatges", "Superfície útil"),
           ("Total habitatges", "Preu mitjà"),
           ("Total habitatges", "Preu m2 útil")])

table117_prov[("Total habitatges", "Municipi")] = np.NaN
table117_prov[("Total habitatges", "País")] = "Catalunya"
    

table117_mun = bbdd_estudi_hab_mod_2021[["PROVINCIA", "Municipi", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["PROVINCIA","Municipi"]).agg({"Municipi":['count'], "Superfície útil": [np.mean], "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).reset_index()

table117_mun.columns = pd.MultiIndex.from_tuples([("Total habitatges", "Província"),
            ("Total habitatges", "Municipi"),
            ("Total habitatges", "Unitats"),
           ("Total habitatges", "Superfície útil"),
           ("Total habitatges", "Preu mitjà"),
           ("Total habitatges", "Preu m2 útil")])
table117_mun[("Total habitatges", "País")] = "Catalunya"




dict80_total = bbdd_estudi_hab_mod_2021[["Superfície útil", "Preu mitjà", "Preu m2 útil"]].mean().to_dict()
dict80_total.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN, "Unitats":bbdd_estudi_hab_mod_2021.shape[0]})

dict80_uni = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["TIPOG"]=="Habitatges Unifamiliars"][["Superfície útil", "Preu mitjà", "Preu m2 útil"]].mean().to_dict()
dict80_uni.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN, "Unitats":bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["TIPOG"]=="Habitatges Unifamiliars"].shape[0]})

dict80_pluri = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["TIPOG"]=="Habitatges Plurifamiliars"][["Superfície útil", "Preu mitjà", "Preu m2 útil"]].mean().to_dict()
dict80_pluri.update({"País":"Catalunya", "Província":np.NaN, "Municipi":np.NaN, "Unitats":bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["TIPOG"]=="Habitatges Plurifamiliars"].shape[0]})

dict80_total_df = pd.DataFrame(dict80_total, index=[0])
dict80_total_df.columns = pd.MultiIndex.from_tuples([
           ("Total habitatges", "Superfície útil"),
           ("Total habitatges", "Preu mitjà"),
           ("Total habitatges", "Preu m2 útil"),
           ("Total habitatges", "País"),
           ("Total habitatges", "Província"),
            ("Total habitatges", "Municipi"),
            ("Total habitatges", "Unitats")])

dict80_uni_df = pd.DataFrame(dict80_uni, index=[0])
dict80_uni_df.columns = pd.MultiIndex.from_tuples([
           ("Habitatges Unifamiliars", "Superfície útil"),
           ("Habitatges Unifamiliars", "Preu mitjà"),
           ("Habitatges Unifamiliars", "Preu m2 útil"),
           ("Habitatges Unifamiliars", "País"),
           ("Habitatges Unifamiliars", "Província"),
            ("Habitatges Unifamiliars", "Municipi"),
            ("Habitatges Unifamiliars", "Unitats")])


dict80_pluri_df = pd.DataFrame(dict80_pluri, index=[0])
dict80_pluri_df.columns = pd.MultiIndex.from_tuples([
           ("Habitatges Plurifamiliars", "Superfície útil"),
           ("Habitatges Plurifamiliars", "Preu mitjà"),
           ("Habitatges Plurifamiliars", "Preu m2 útil"),
           ("Habitatges Plurifamiliars", "País"),
           ("Habitatges Plurifamiliars", "Província"),
            ("Habitatges Plurifamiliars", "Municipi"),
            ("Habitatges Plurifamiliars", "Unitats")])

table117_total = pd.concat([table117_prov, table117_mun, dict80_total_df], axis=0)
table117_pluri = pd.concat([table117pluri_prov, table117pluri_mun, dict80_pluri_df], axis=0)
table117_uni = pd.concat([table117uni_prov, table117uni_mun, dict80_uni_df], axis=0)

table117_2021 = table117_total.copy()

table117_2021.columns = pd.MultiIndex.from_tuples([
               ("Total habitatges", "Província"),
                           ("Total habitatges", "Unitats 2021"),
           ("Total habitatges", "Superfície útil 2021"),
           ("Total habitatges", "Preu mitjà 2021"),
           ("Total habitatges", "Preu m2 útil 2021"),
                       ("Total habitatges", "Municipi"),
                       ("Total habitatges", "País")])
table117_2022 = table80_total.copy()
table117_2022.columns = pd.MultiIndex.from_tuples([
               ("Total habitatges", "Província"),
                           ("Total habitatges", "Unitats 2022"),
           ("Total habitatges", "Superfície útil 2022"),
           ("Total habitatges", "Preu mitjà 2022"),
           ("Total habitatges", "Preu m2 útil 2022"),
                       ("Total habitatges", "Municipi"),
                       ("Total habitatges", "País")])

table117 = pd.merge(table117_2022, table117_2021, how="outer", on=[("Total habitatges", "País"), ("Total habitatges", "Província"), ("Total habitatges", "Municipi")])
table117[("Total habitatges","Variació unitats")] = table117[("Total habitatges", "Unitats 2022")] - table117[("Total habitatges", "Unitats 2021")]
table117[("Total habitatges","Variació Superfície útil")] = table117[("Total habitatges", "Superfície útil 2022")] - table117[("Total habitatges", "Superfície útil 2021")]
table117[("Total habitatges","Variació Preu mitjà")] = table117[("Total habitatges", "Preu mitjà 2022")] - table117[("Total habitatges", "Preu mitjà 2021")]
table117[("Total habitatges","Variació Preu m2 útil")] = table117[("Total habitatges", "Preu m2 útil 2022")] - table117[("Total habitatges", "Preu m2 útil 2021")]

table117[("Total habitatges","Variació % unitats")] = ((table117[("Total habitatges", "Unitats 2022")]/table117[("Total habitatges", "Unitats 2021")])-1)*100
table117[("Total habitatges","Variació % Superfície útil")] = ((table117[("Total habitatges", "Superfície útil 2022")]/table117[("Total habitatges", "Superfície útil 2021")])-1)*100
table117[("Total habitatges","Variació % Preu mitjà")] = ((table117[("Total habitatges", "Preu mitjà 2022")]/table117[("Total habitatges", "Preu mitjà 2021")])-1)*100
table117[("Total habitatges","Variació % Preu m2 útil")] = ((table117[("Total habitatges", "Preu m2 útil 2022")]/table117[("Total habitatges", "Preu m2 útil 2021")])-1)*100



PESTANYA 50 - 53

In [44]:
table121_2021 = table117_uni.copy()

table121_2021.columns = pd.MultiIndex.from_tuples([
               ("Habitatges Unifamiliars", "Província"),
                           ("Habitatges Unifamiliars", "Unitats 2021"),
           ("Habitatges Unifamiliars", "Superfície útil 2021"),
           ("Habitatges Unifamiliars", "Preu mitjà 2021"),
           ("Habitatges Unifamiliars", "Preu m2 útil 2021"),
                       ("Habitatges Unifamiliars", "Municipi"),
                       ("Habitatges Unifamiliars", "País")])
table121_2022 = table80_uni.copy()
table121_2022.columns = pd.MultiIndex.from_tuples([
               ("Habitatges Unifamiliars", "Província"),
                           ("Habitatges Unifamiliars", "Unitats 2022"),
           ("Habitatges Unifamiliars", "Superfície útil 2022"),
           ("Habitatges Unifamiliars", "Preu mitjà 2022"),
           ("Habitatges Unifamiliars", "Preu m2 útil 2022"),
                       ("Habitatges Unifamiliars", "Municipi"),
                       ("Habitatges Unifamiliars", "País")])

table121 = pd.merge(table121_2021, table121_2022, how="outer", on=[("Habitatges Unifamiliars", "País"), ("Habitatges Unifamiliars", "Província"), ("Habitatges Unifamiliars", "Municipi")])
table121[("Habitatges Unifamiliars","Variació unitats")] = table121[("Habitatges Unifamiliars", "Unitats 2022")] - table121[("Habitatges Unifamiliars", "Unitats 2021")]
table121[("Habitatges Unifamiliars","Variació Superfície útil")] = table121[("Habitatges Unifamiliars", "Superfície útil 2022")] - table121[("Habitatges Unifamiliars", "Superfície útil 2021")]
table121[("Habitatges Unifamiliars","Variació Preu mitjà")] = table121[("Habitatges Unifamiliars", "Preu mitjà 2022")] - table121[("Habitatges Unifamiliars", "Preu mitjà 2021")]
table121[("Habitatges Unifamiliars","Variació Preu m2 útil")] = table121[("Habitatges Unifamiliars", "Preu m2 útil 2022")] - table121[("Habitatges Unifamiliars", "Preu m2 útil 2021")]

table121[("Habitatges Unifamiliars","Variació % unitats")] = ((table121[("Habitatges Unifamiliars", "Unitats 2022")]/table121[("Habitatges Unifamiliars", "Unitats 2021")])-1)*100
table121[("Habitatges Unifamiliars","Variació % Superfície útil")] = ((table121[("Habitatges Unifamiliars", "Superfície útil 2022")]/table121[("Habitatges Unifamiliars", "Superfície útil 2021")])-1)*100
table121[("Habitatges Unifamiliars","Variació % Preu mitjà")] = ((table121[("Habitatges Unifamiliars", "Preu mitjà 2022")]/table121[("Habitatges Unifamiliars", "Preu mitjà 2021")])-1)*100
table121[("Habitatges Unifamiliars","Variació % Preu m2 útil")] = ((table121[("Habitatges Unifamiliars", "Preu m2 útil 2022")]/table121[("Habitatges Unifamiliars", "Preu m2 útil 2021")])-1)*100

table121

Habitatges Unifamiliars                                                    \
                 Província Unitats 2021 Superfície útil 2021 Preu mitjà 2021   
0                Barcelona        175.0           183.574611   607586.971429   
1                   Girona         52.0           155.335985   505442.307692   
2                   Lleida         13.0           134.875849   339961.538462   
3                Tarragona         91.0           160.586343   385637.362637   
4                Barcelona          4.0           147.120000   439000.000000   
..                     ...          ...                  ...             ...   
59                  Girona          NaN                  NaN             NaN   
60                  Girona          NaN                  NaN             NaN   
61                  Lleida          NaN                  NaN             NaN   
62               Tarragona          NaN                  NaN             NaN   
63               Tarragona          NaN                  NaN             NaN   

                                                             \
   Preu m2 útil 2021       Municipi       País Unitats 2022   
0        3355.089214            NaN  Catalunya        170.0   
1        3045.099837            NaN  Catalunya         64.0   
2        2569.478556            NaN  Catalunya         18.0   
3        2528.371313            NaN  Catalunya         86.0   
4        2983.958673  Arenys de Mar  Catalunya          6.0   
..               ...            ...        ...          ...   
59               NaN         Girona  Catalunya          1.0   
60               NaN  Lloret de Mar  Catalunya          4.0   
61               NaN     Mollerussa  Catalunya          7.0   
62               NaN        Amposta  Catalunya         26.0   
63               NaN       Calafell  Catalunya          2.0   

                                                                            \
   Superfície útil 2022 Preu mitjà 2022 Preu m2 útil 2022 Variació unitats   
0            161.355786   637229.411765       3813.597110             -5.0   
1            131.044893   404349.609375       3179.744156             12.0   
2            131.707358   450416.666667       3326.504336              5.0   
3            167.166534   370784.534884       2377.386291             -5.0   
4            149.016667   710000.000000       4845.093292              2.0   
..                  ...             ...               ...              ...   
59           200.000000   695000.000000       3475.000000              NaN   
60           118.940000   685000.000000       5759.206323              NaN   
61           126.160000   395000.000000       3130.944832              NaN   
62           202.923361   282307.692308       1401.682250              NaN   
63           157.896000   365000.000000       2311.648173              NaN   

                                                                       \
   Variació Superfície útil Variació Preu mitjà Variació Preu m2 útil   
0                -22.218825        29642.440336            458.507896   
1                -24.291093      -101092.698317            134.644319   
2                 -3.168492       110455.128205            757.025780   
3                  6.580191       -14852.827754           -150.985022   
4                  1.896667       271000.000000           1861.134619   
..                      ...                 ...                   ...   
59                      NaN                 NaN                   NaN   
60                      NaN                 NaN                   NaN   
61                      NaN                 NaN                   NaN   
62                      NaN                 NaN                   NaN   
63                      NaN                 NaN                   NaN   

                                                                        \
   Variació % unitats Variació % Superfície útil Variació % Preu mitjà   
0           -2.857143        

PESTANYA 54 - 57

In [45]:
table125_2021 = table117_pluri.copy()

table125_2021.columns = pd.MultiIndex.from_tuples([
               ("Habitatges Plurifamiliars", "Província"),
                           ("Habitatges Plurifamiliars", "Unitats 2021"),
           ("Habitatges Plurifamiliars", "Superfície útil 2021"),
           ("Habitatges Plurifamiliars", "Preu mitjà 2021"),
           ("Habitatges Plurifamiliars", "Preu m2 útil 2021"),
                       ("Habitatges Plurifamiliars", "Municipi"),
                       ("Habitatges Plurifamiliars", "País")])
table125_2022 = table80_pluri.copy()
table125_2022.columns = pd.MultiIndex.from_tuples([
               ("Habitatges Plurifamiliars", "Província"),
                           ("Habitatges Plurifamiliars", "Unitats 2022"),
           ("Habitatges Plurifamiliars", "Superfície útil 2022"),
           ("Habitatges Plurifamiliars", "Preu mitjà 2022"),
           ("Habitatges Plurifamiliars", "Preu m2 útil 2022"),
                       ("Habitatges Plurifamiliars", "Municipi"),
                       ("Habitatges Plurifamiliars", "País")])

table125 = pd.merge(table125_2021, table125_2022,how="outer", on=[("Habitatges Plurifamiliars", "País"), ("Habitatges Plurifamiliars", "Província"), ("Habitatges Plurifamiliars", "Municipi")])
table125[("Habitatges Plurifamiliars","Variació unitats")] = table125[("Habitatges Plurifamiliars", "Unitats 2022")] - table125[("Habitatges Plurifamiliars", "Unitats 2021")]
table125[("Habitatges Plurifamiliars","Variació Superfície útil")] = table125[("Habitatges Plurifamiliars", "Superfície útil 2022")] - table125[("Habitatges Plurifamiliars", "Superfície útil 2021")]
table125[("Habitatges Plurifamiliars","Variació Preu mitjà")] = table125[("Habitatges Plurifamiliars", "Preu mitjà 2022")] - table125[("Habitatges Plurifamiliars", "Preu mitjà 2021")]
table125[("Habitatges Plurifamiliars","Variació Preu m2 útil")] = table125[("Habitatges Plurifamiliars", "Preu m2 útil 2022")] - table125[("Habitatges Plurifamiliars", "Preu m2 útil 2021")]

table125[("Habitatges Plurifamiliars","Variació % unitats")] = ((table125[("Habitatges Plurifamiliars", "Unitats 2022")]/table125[("Habitatges Plurifamiliars", "Unitats 2021")])-1)*100
table125[("Habitatges Plurifamiliars","Variació % Superfície útil")] = ((table125[("Habitatges Plurifamiliars", "Superfície útil 2022")]/table125[("Habitatges Plurifamiliars", "Superfície útil 2021")])-1)*100
table125[("Habitatges Plurifamiliars","Variació % Preu mitjà")] = ((table125[("Habitatges Plurifamiliars", "Preu mitjà 2022")]/table125[("Habitatges Plurifamiliars", "Preu mitjà 2021")])-1)*100
table125[("Habitatges Plurifamiliars","Variació % Preu m2 útil")] = ((table125[("Habitatges Plurifamiliars", "Preu m2 útil 2022")]/table125[("Habitatges Plurifamiliars", "Preu m2 útil 2021")])-1)*100

table125

Habitatges Plurifamiliars                                    \
                   Província Unitats 2021 Superfície útil 2021   
0                  Barcelona         4703            80.560621   
1                     Girona          481            79.221652   
2                     Lleida          251            84.666668   
3                  Tarragona          528            74.168252   
4                  Barcelona           48            80.339682   
..                       ...          ...                  ...   
93                 Tarragona           53            68.003114   
94                 Tarragona            2            58.223000   
95                 Tarragona           26            57.055527   
96                 Tarragona           24            81.284669   
97                       NaN         5963            80.059431   

                                                                             \
   Preu mitjà 2021 Preu m2 útil 2021       Municipi       País Unitats 2022   
0    405380.610461       4929.691621            NaN  Catalunya       6020.0   
1    301718.349272       3794.745376            NaN  Catalunya        918.0   
2    188189.888446       2160.843198            NaN  Catalunya        299.0   
3    183452.287879       2402.983055            NaN  Catalunya        606.0   
4    284557.500000       3709.451304  Arenys de Mar  Catalunya         70.0   
..             ...               ...            ...        ...          ...   
93    80460.226415       1152.847132        Tortosa  Catalunya         19.0   
94    43900.000000        754.310815          Valls  Catalunya          6.0   
95    74219.230769       1319.416733   Vendrell, el  Catalunya         35.0   
96   200333.333333       2450.841908      Vila-seca  Catalunya         22.0   
97   368225.726480       4497.863309            NaN  Catalunya       7843.0   

                                                                            \
   Superfície útil 2022 Preu mitjà 2022 Preu m2 útil 2022 Variació unitats   
0             77.308440   382726.480565       4824.044249           1317.0   
1             79.102508   374048.692810       4789.002760            437.0   
2             83.773325   195843.247492       2292.478696             48.0   
3             73.636430   226195.125413       3040.310083             78.0   
4             75.917668   252178.000000       3456.843449             22.0   
..                  ...             ...               ...              ...   
93            68.070522   121161.894737       1745.531657            -34.0   
94            35.833333    45166.666667       1263.924851              4.0   
95            75.579833   108542.857143       1429.910548              9.0   
96            82.367878   210181.818182       2552.653940             -2.0   
97            77.481170   362491.583578       4585.608817           1880.0   

                                                                       \
   Variació Superfície útil Variació Preu mitjà Variació Preu m2 útil   
0                 -3.252181       -22654.129897           -105.647372   
1                 -0.119144        72330.343538            994.257384   
2                 -0.893343         7653.359045            131.635498   
3                 -0.531821        42742.837534            637.327028   
4                 -4.422014       -32379.500000           -252.607855   
..                      ...                 ...                   ...   
93                 0.067408        40701.668322            592.684525   
94               -22.389667         1266.666667            509.614037   
95                18.524307        34323.626374            110.493815   
96                 1.083209         9848.484848            101.812032   
97                -2.578261        -5734.142902             87.745508   

                                                                        \
   Variació % unitats Variació % Superfície útil Variació % Preu mitjà   
0  

PESTANYA 59

In [46]:
def order_columns_dis(table_n):
    cols = ["Municipi", "Districte"] + [col for col in table_n.columns if col not in ["Municipi", "Districte"]]
    table_n = table_n.reindex(columns=cols)
    return(table_n)

In [47]:
table134 = bbdd_estudi_prom[["Nom DIST", "HABIT", "HABIV", "HABIP"]].groupby("Nom DIST").sum().reset_index()
table134.columns = ["Districte", "Habitatges a la promoció", "Habitatges venuts","Habitatges en oferta"]
table134["Municipi"] = "Barcelona"

table134prom_dis = bbdd_estudi_prom[["Nom DIST"]].value_counts().reset_index().rename(columns={"Nom DIST":"Districte",0:"Promocions"})
table134prom_dis["Municipi"] = "Barcelona"

table134 = pd.merge(table134, table134prom_dis, how="left", on=["Municipi", "Districte"])
table134 = order_columns_dis(table134)

table134_bcn = table11[table11["Municipi"]=="Barcelona"].drop(["País", "Província"], axis=1).copy()

table134_bcn["Habitatges venuts"] = table134["Habitatges a la promoció"] - table134["Habitatges en oferta"]

table134 = pd.concat([table134, table134_bcn], axis=0)
table134["Mitjana (en oferta)"] = table134["Habitatges en oferta"]/table134["Promocions"]
table134["Mitjana (a la promoció)"] = table134["Habitatges a la promoció"]/table134["Promocions"]
table134["% habitatges venuts"] = table134["Habitatges venuts"].mul(100)/table134["Habitatges a la promoció"]
table134["% habitatges en venda"] = table134["Habitatges en oferta"].mul(100)/table134["Habitatges a la promoció"]

table134

,Municipi,Districte,Habitatges a la promoció,Habitatges venuts,Habitatges en oferta,Promocions,Mitjana (en oferta),Mitjana (a la promoció),% habitatges en venda,% habitatges venuts
0,Barcelona,01 Ciutat Vella,292,121,171,21,8.142857,13.904762,58.561644,41.438356
1,Barcelona,02 Eixample,598,342,256,31,8.258065,19.290323,42.809365,57.190635
2,Barcelona,03 Sants-Montjuïc,265,174,91,15,6.066667,17.666667,34.339623,65.660377
3,Barcelona,04 Les Corts,133,107,26,7,3.714286,19.000000,19.548872,80.451128
4,Barcelona,05 Sarrià - Sant Gervasi,150,66,84,14,6.000000,10.714286,56.000000,44.000000
5,Barcelona,06 Gràcia,245,114,131,18,7.277778,13.611111,53.469388,46.530612
6,Barcelona,07 Horta-Guinardó,436,247,189,35,5.400000,12.457143,43.348624,56.651376
7,Barcelona,08 Nou Barris,87,39,48,7,6.857143,12.428571,55.172414,44.827586
8,Barcelona,09 Sant Andreu,395,173,222,24,9.250000,16.458333,56.202532,43.797468
9,Barcelona,10 Sant Martí,677,505,172,27,6.370370,25.074074,25.406204,74.593796


In [48]:
table136_dis = bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Barcelona"][["Nom DIST", "HABIP", "HABIV","HABIT"]].groupby("Nom DIST").sum().reset_index()
table136_barri = bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Barcelona"][["Nom DIST","Nom BARRI", "HABIP","HABIV", "HABIT"]].groupby(["Nom DIST","Nom BARRI"]).sum().reset_index()

table136_hab = table136_barri.append(table136_dis, ignore_index=True)
table136_hab["Municipi"] = "Barcelona"

table136_hab = table136_hab.append({"Municipi":"Barcelona", "Nom DIST": np.NaN, "Nom BARRI":np.NaN, "HABIP":bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Barcelona"]["HABIP"].sum(), "HABIT":bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Barcelona"]["HABIT"].sum(), "HABIV":bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Barcelona"]["HABIV"].sum()}, ignore_index=True)
table136_hab = table136_hab.rename(columns={"HABIV":"Habitatges venuts","HABIP":"Habitatges en oferta", "HABIT":"Habitatges a la promoció","Nom DIST":"Districte"})
table136_hab = table136_hab[["Municipi", "Districte", "Nom BARRI","Habitatges venuts", "Habitatges en oferta", "Habitatges a la promoció"]]


table136prom_dis = bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Barcelona"][["Nom DIST"]].value_counts().reset_index().rename(columns={0:"Promocions"})
table136prom_dis["Nom BARRI"] = np.NaN
table136prom_mun = bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Barcelona"][["Nom DIST", "Nom BARRI"]].value_counts().reset_index().rename(columns = {0:"Promocions"})
table136prom = pd.concat([table136prom_mun,table136prom_dis], axis=0)
table136prom["Municipi"] = "Barcelona"

table136prom = table136prom.append({"Municipi":"Barcelona", "Nom DIST": np.NaN, "Nom BARRI":np.NaN, "Promocions":bbdd_estudi_prom[bbdd_estudi_prom["Municipi"]=="Barcelona"].shape[0]}, ignore_index=True)
table136prom = table136prom.rename(columns={"Nom DIST":"Districte"}).sort_values(["Municipi", "Districte", "Nom BARRI"], ascending= True)
table136 = pd.merge(table136prom, table136_hab, how="left", on =["Municipi", "Districte", "Nom BARRI"])
table136 = table136[["Municipi", "Districte", "Nom BARRI", "Promocions", "Habitatges venuts", "Habitatges en oferta", "Habitatges a la promoció"]].rename(columns={"Nom BARRI":"Barri"})

table136["Mitjana (en oferta)"] = table136["Habitatges en oferta"]/table136["Promocions"]
table136["Mitjana (a la promoció)"] = table136["Habitatges a la promoció"]/table136["Promocions"]
table136["% habitatges venuts"] = table136["Habitatges venuts"].mul(100)/table136["Habitatges a la promoció"]
table136["% habitatges en venda"] = table136["Habitatges en oferta"].mul(100)/table136["Habitatges a la promoció"]
table136

,Municipi,Districte,Barri,Promocions,Habitatges venuts,Habitatges en oferta,Habitatges a la promoció,Mitjana (en oferta),Mitjana (a la promoció),% habitatges venuts,% habitatges en venda
0,Barcelona,01 Ciutat Vella,1 el Raval,7,85,47,132,6.714286,18.857143,64.393939,35.606061
1,Barcelona,01 Ciutat Vella,2 el Barri Gòtic,9,27,82,109,9.111111,12.111111,24.770642,75.229358
2,Barcelona,01 Ciutat Vella,"4 Sant Pere, Santa Caterina i la Ribera",5,9,42,51,8.400000,10.200000,17.647059,82.352941
3,Barcelona,01 Ciutat Vella,NaN,21,121,171,292,8.142857,13.904762,41.438356,58.561644
4,Barcelona,02 Eixample,10 Sant Antoni,2,0,16,16,8.000000,8.000000,0.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...
58,Barcelona,10 Sant Martí,67 la Vila Olímpica del Poblenou,2,26,19,45,9.500000,22.500000,57.777778,42.222222
59,Barcelona,10 Sant Martí,68 el Poblenou,6,134,14,148,2.333333,24.666667,90.540541,9.459459
60,Barcelona,10 Sant Martí,69 Diagonal Mar i el Front Marítim del Poblenou,2,7,15,22,7.500000,11.000000,31.818182,68.181818
61,Barcelona,10 Sant Martí,NaN,27,505,172,677,6.370370,25.074074,74.593796,25.406204


PESTANYA 60 - 62

In [49]:
table151_aux = pd.concat([bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi","Total dormitoris"]].value_counts().reset_index(),
bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi", "Nom DIST","Total dormitoris"]].value_counts().reset_index(),
bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI","Total dormitoris"]].value_counts().reset_index()], axis=0).rename(columns={"Nom DIST":"Districte", "Nom BARRI": "Barri",0:"Total"})
table151_aux= table151_aux.pivot(index=["Municipi", "Districte", "Barri"], columns='Total dormitoris', values='Total')


table151 = table151_aux.apply(lambda x: x.mul(100)/x.sum(), axis=1).reset_index()
table151.columns = ['Municipi', 'Districte', 'Barri', '% 0D', '% 1D', '% 2D', '% 3D', '% 4D', '% 5+D']

table151_aux["Total"] = table151_aux.sum(axis=1)

table151 = pd.merge(table151_aux.reset_index(), table151, how="left", on=["Municipi", "Districte", "Barri"])
table151

,Municipi,Districte,Barri,0D,1D,2D,3D,4D,5+D,Total,% 0D,% 1D,% 2D,% 3D,% 4D,% 5+D
0,Barcelona,NaN,NaN,25.0,167.0,579.0,493.0,113.0,13.0,1390.0,1.798561,12.014388,41.654676,35.467626,8.129496,0.935252
1,Barcelona,01 Ciutat Vella,NaN,NaN,42.0,91.0,37.0,1.0,NaN,171.0,NaN,24.561404,53.216374,21.637427,0.584795,NaN
2,Barcelona,01 Ciutat Vella,1 el Raval,NaN,15.0,29.0,3.0,NaN,NaN,47.0,NaN,31.914894,61.702128,6.382979,NaN,NaN
3,Barcelona,01 Ciutat Vella,2 el Barri Gòtic,NaN,24.0,32.0,25.0,1.0,NaN,82.0,NaN,29.268293,39.024390,30.487805,1.219512,NaN
4,Barcelona,01 Ciutat Vella,"4 Sant Pere, Santa Caterina i la Ribera",NaN,3.0,30.0,9.0,NaN,NaN,42.0,NaN,7.142857,71.428571,21.428571,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Barcelona,10 Sant Martí,65 el Clot,NaN,NaN,6.0,10.0,3.0,NaN,19.0,NaN,NaN,31.578947,52.631579,15.789474,NaN
59,Barcelona,10 Sant Martí,66 el Parc i la Llacuna del Poblenou,22.0,NaN,NaN,18.0,11.0,NaN,51.0,43.137255,NaN,NaN,35.294118,21.568627,NaN
60,Barcelona,10 Sant Martí,67 la Vila Olímpica del Poblenou,NaN,10.0,2.0,7.0,NaN,NaN,19.0,NaN,52.631579,10.526316,36.842105,NaN,NaN
61,Barcelona,10 Sant Martí,68 el Poblenou,NaN,NaN,4.0,4.0,6.0,NaN,14.0,NaN,NaN,28.571429,28.571429,42.857143,NaN


PESTANYA 63 - 65

In [50]:
table190tipo_mun = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "TIPOG"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table190tipo_mun.columns = table190tipo_mun.columns.droplevel(level=1)
table190tipo_dis = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "Nom DIST","TIPOG"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table190tipo_dis.columns = table190tipo_dis.columns.droplevel(level=1)

table190tipo_barri = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "Nom DIST","Nom BARRI","TIPOG"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table190tipo_barri.columns = table190tipo_barri.columns.droplevel(level=1)


table190total_mun = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table190total_mun.columns = table190total_mun.columns.droplevel(level=1)
table190total_mun["TIPOG"]= "Total habitatges"

table190total_dis = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "Nom DIST"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table190total_dis.columns = table190total_dis.columns.droplevel(level=1)
table190total_dis["TIPOG"]= "Total habitatges"

table190total_barri = bbdd_estudi_hab_mod[bbdd_estudi_hab_mod["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "Nom DIST","Nom BARRI"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table190total_barri.columns = table190total_barri.columns.droplevel(level=1)

table190total_barri["TIPOG"]= "Total habitatges"


table190 = pd.concat([table190total_mun, table190total_dis, table190total_barri, table190tipo_mun, table190tipo_dis, table190tipo_barri], axis=0).rename(columns={"Nom DIST":"Districte", "Nom BARRI":"Barri"})
table190 = pd.melt(table190, id_vars=["Municipi", "Districte", "Barri", "TIPOG"], value_vars=["Unitats","Superfície útil", "Preu mitjà", "Preu m2 útil"],value_name="Valor").rename(columns={0:"Variable"})

table190 = table190.pivot(index=["Municipi", "Districte", "Barri"], columns=["TIPOG", "Variable"], values=["Valor"])
table190.columns = table190.columns.droplevel(level=0)
table190.sort_index(axis=1, level=0, sort_remaining=False, inplace=True)
table190

TIPOG                                                                     Habitatges Plurifamiliars  \
Variable                                                                                    Unitats   
Municipi  Districte       Barri                                                                       
Barcelona NaN             NaN                                                                1380.0   
          01 Ciutat Vella NaN                                                                 171.0   
                          1 el Raval                                                           47.0   
                          2 el Barri Gòtic                                                     82.0   
                          4 Sant Pere, Santa Caterina i la Ribera                              42.0   
...                                                                                             ...   
          10 Sant Martí   65 el Clot                                                           19.0   
                          66 el Parc i la Llacuna del Poblenou                                 51.0   
                          67 la Vila Olímpica del Poblenou                                     19.0   
                          68 el Poblenou                                                       14.0   
                          69 Diagonal Mar i el Front Marítim del Poblenou                      15.0   

TIPOG                                                                                      \
Variable                                                                  Superfície útil   
Municipi  Districte       Barri                                                             
Barcelona NaN             NaN                                                   80.053614   
          01 Ciutat Vella NaN                                                   81.078771   
                          1 el Raval                                            62.202690   
                          2 el Barri Gòtic                                      87.559816   
                          4 Sant Pere, Santa Caterina i la Ribera               89.548536   
...                                                                                   ...   
          10 Sant Martí   65 el Clot                                            87.668564   
                          66 el Parc i la Llacuna del Poblenou                  85.806174   
                          67 la Vila Olímpica del Poblenou                      69.252363   
                          68 el Poblenou                                        71.818058   
                          69 Diagonal Mar i el Front Marítim del Poblenou       68.645807   

TIPOG                                                                                     \
Variable                                                                      Preu mitjà   
Municipi  Districte       Barri                                                            
Barcelona NaN             NaN                                              629114.031884   
          01 Ciutat Vella NaN                                              678719.298246   
                          1 el Raval                                       427425.531915   
                          2 el Barri Gòtic                                 739231.707317   
                          4 Sant Pere, Santa Caterina i la Ribera          841785.714286   
...                                                                                  ...   
          10 Sant Martí   65 el Clot                                       483761.894737   
                          66 el Parc i la Llacuna del Poblenou             598729.411765   
                          67 la Vila Olímpica del Poblenou                 409368.421053   
                          68 el Poblenou                                   519103.357143   
                          69 Diagonal Mar i el Front Marít

PESTANYA 67

In [51]:
table205_dis = bbdd_estudi_prom_2021[bbdd_estudi_prom_2021["Municipi"]=="Barcelona"][["Nom DIST", "HABIP", "HABIV","HABIT"]].groupby("Nom DIST").sum().reset_index()
table205_barri = bbdd_estudi_prom_2021[bbdd_estudi_prom_2021["Municipi"]=="Barcelona"][["Nom DIST","Nom BARRI", "HABIP","HABIV", "HABIT"]].groupby(["Nom DIST","Nom BARRI"]).sum().reset_index()

table205_hab = table205_barri.append(table205_dis, ignore_index=True)
table205_hab["Municipi"] = "Barcelona"

table205_hab = table205_hab.append({"Municipi":"Barcelona", "Nom DIST": np.NaN, "Nom BARRI":np.NaN, "HABIP":bbdd_estudi_prom_2021[bbdd_estudi_prom_2021["Municipi"]=="Barcelona"]["HABIP"].sum(), "HABIT":bbdd_estudi_prom_2021[bbdd_estudi_prom_2021["Municipi"]=="Barcelona"]["HABIT"].sum(), "HABIV":bbdd_estudi_prom_2021[bbdd_estudi_prom_2021["Municipi"]=="Barcelona"]["HABIV"].sum()}, ignore_index=True)
table205_hab = table205_hab.rename(columns={"HABIV":"Habitatges venuts","HABIP":"Habitatges en oferta", "HABIT":"Habitatges a la promoció","Nom DIST":"Districte"})
table205_hab = table205_hab[["Municipi", "Districte", "Nom BARRI","Habitatges venuts", "Habitatges en oferta", "Habitatges a la promoció"]]


table205prom_dis = bbdd_estudi_prom_2021[bbdd_estudi_prom_2021["Municipi"]=="Barcelona"][["Nom DIST"]].value_counts().reset_index().rename(columns={0:"Promocions"})
table205prom_dis["Nom BARRI"] = np.NaN
table205prom_mun = bbdd_estudi_prom_2021[bbdd_estudi_prom_2021["Municipi"]=="Barcelona"][["Nom DIST", "Nom BARRI"]].value_counts().reset_index().rename(columns = {0:"Promocions"})
table205prom = pd.concat([table205prom_mun,table205prom_dis], axis=0)
table205prom["Municipi"] = "Barcelona"

table205prom = table205prom.append({"Municipi":"Barcelona", "Nom DIST": np.NaN, "Nom BARRI":np.NaN, "Promocions":bbdd_estudi_prom_2021[bbdd_estudi_prom_2021["Municipi"]=="Barcelona"].shape[0]}, ignore_index=True)
table205prom = table205prom.rename(columns={"Nom DIST":"Districte"}).sort_values(["Municipi", "Districte", "Nom BARRI"], ascending= True)
table205 = pd.merge(table205prom, table205_hab, how="left", on =["Municipi", "Districte", "Nom BARRI"])
table205_aux = table205[["Municipi", "Districte", "Nom BARRI", "Promocions", "Habitatges venuts", "Habitatges en oferta", "Habitatges a la promoció"]].rename(columns={"Nom BARRI":"Barri"})

table205_aux["Mitjana (en oferta)"] = table205_aux["Habitatges en oferta"]/table205_aux["Promocions"]
table205_aux["Mitjana (a la promoció)"] = table205_aux["Habitatges a la promoció"]/table205_aux["Promocions"]
table205_aux["% habitatges venuts"] = table205_aux["Habitatges venuts"].mul(100)/table205_aux["Habitatges a la promoció"]
table205_aux["% habitatges en venda"] = table205_aux["Habitatges en oferta"].mul(100)/table205_aux["Habitatges a la promoció"]
table205_2021 = table205_aux[["Municipi", "Districte", "Barri", "Promocions", "Habitatges en oferta"]].rename(columns= {"Promocions":"Promocions 2021", "Habitatges en oferta":"Habitatges en oferta 2021"}).copy()
table205_2022 = table136[["Municipi", "Districte", "Barri", "Promocions", "Habitatges en oferta"]].rename(columns= {"Promocions":"Promocions 2022", "Habitatges en oferta":"Habitatges en oferta 2022"}).copy()
table205 = pd.merge(table205_2021, table205_2022, how="left", on=["Municipi", "Districte", "Barri"]).sort_values(["Municipi", "Districte", "Barri"], ascending=True)
table205["Variació promocions"] = table205["Promocions 2022"] - table205["Promocions 2021"]
table205["Variació habitatges"] = table205["Habitatges en oferta 2022"] - table205["Habitatges en oferta 2021"]
table205


,Municipi,Districte,Barri,Promocions 2021,Habitatges en oferta 2021,Promocions 2022,Habitatges en oferta 2022,Variació promocions,Variació habitatges
0,Barcelona,01 Ciutat Vella,1 el Raval,4,19,7.0,47.0,3.0,28.0
1,Barcelona,01 Ciutat Vella,2 el Barri Gòtic,8,49,9.0,82.0,1.0,33.0
2,Barcelona,01 Ciutat Vella,3 la Barceloneta,1,1,NaN,NaN,NaN,NaN
3,Barcelona,01 Ciutat Vella,"4 Sant Pere, Santa Caterina i la Ribera",7,45,5.0,42.0,-2.0,-3.0
4,Barcelona,01 Ciutat Vella,NaN,20,114,21.0,171.0,1.0,57.0
...,...,...,...,...,...,...,...,...,...
59,Barcelona,10 Sant Martí,68 el Poblenou,10,59,6.0,14.0,-4.0,-45.0
60,Barcelona,10 Sant Martí,69 Diagonal Mar i el Front Marítim del Poblenou,2,14,2.0,15.0,0.0,1.0
61,Barcelona,10 Sant Martí,70 el Besòs i el Maresme,4,8,NaN,NaN,NaN,NaN
62,Barcelona,10 Sant Martí,NaN,45,264,27.0,172.0,-18.0,-92.0


PESTANYA 208

In [52]:
table208tipo_mun = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "TIPOG"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table208tipo_mun.columns = table208tipo_mun.columns.droplevel(level=1)
table208tipo_dis = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "Nom DIST","TIPOG"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table208tipo_dis.columns = table208tipo_dis.columns.droplevel(level=1)

table208tipo_barri = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "TIPOG", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "Nom DIST","Nom BARRI","TIPOG"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table208tipo_barri.columns = table208tipo_barri.columns.droplevel(level=1)


table208total_mun = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table208total_mun.columns = table208total_mun.columns.droplevel(level=1)
table208total_mun["TIPOG"]= "Total habitatges"

table208total_dis = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "Nom DIST"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table208total_dis.columns = table208total_dis.columns.droplevel(level=1)
table208total_dis["TIPOG"]= "Total habitatges"

table208total_barri = bbdd_estudi_hab_mod_2021[bbdd_estudi_hab_mod_2021["Municipi"]=="Barcelona"][["Municipi", "Nom DIST", "Nom BARRI", "Superfície útil", "Preu mitjà", "Preu m2 útil"]].groupby(["Municipi", "Nom DIST","Nom BARRI"]).agg({"Municipi":['count'], "Superfície útil": [np.mean],
                                                                                    "Preu mitjà": [np.mean], "Preu m2 útil": [np.mean]}).rename(columns={"Municipi":"Unitats"}).reset_index()
table208total_barri.columns = table208total_barri.columns.droplevel(level=1)

table208total_barri["TIPOG"]= "Total habitatges"


table208_2021 = pd.concat([table208total_mun, table208total_dis, table208total_barri, table208tipo_mun, table208tipo_dis, table208tipo_barri], axis=0).rename(columns={"Nom DIST":"Districte", "Nom BARRI":"Barri"})
table208_2021 = pd.melt(table208_2021, id_vars=["Municipi", "Districte", "Barri", "TIPOG"], value_vars=["Unitats","Superfície útil", "Preu mitjà", "Preu m2 útil"],value_name="Valor").rename(columns={0:"Variable"})

table208_2021 = table208_2021.pivot(index=["Municipi", "Districte", "Barri"], columns=["TIPOG", "Variable"], values=["Valor"])
table208_2021.columns = table208_2021.columns.droplevel(level=0)
table208_2021.sort_index(axis=1, level=0, sort_remaining=False, inplace=True)


col_names = {col: col + " 2021" for col in table208_2021.columns.get_level_values(1)}
table208_2021.rename(columns=col_names, level=1, inplace=True)

table208_2022 = table190.copy()
col_names = {col: col + " 2022" for col in table208_2022.columns.get_level_values(1)}
table208_2022.rename(columns=col_names, level=1, inplace=True)

table208 = pd.concat([table208_2022, table208_2021], axis=1)

list_1 = ["Total habitatges", "Habitatges Unifamiliars", "Habitatges Plurifamiliars"]
list_2 = ["Unitats", "Superfície útil", "Preu mitjà", "Preu m2 útil"]

for i in ["Total habitatges", "Habitatges Unifamiliars", "Habitatges Plurifamiliars"]:
    for j in ["Unitats", "Superfície útil", "Preu mitjà", "Preu m2 útil"]:
        table208[(i,j + " var")] = (table208[(i,j + " 2022")]-table208[(i,j + " 2021")])
        table208[(i,j + " % var")] = ((table208[(i,j + " 2022")]/table208[(i,j + " 2021")])-1)*100


table208 = table208.xs('Total habitatges', level=0, axis=1)
table208 = table208.sort_index(axis=1).reset_index()
table208[table208["Barri"].isna()]



Variable,Municipi,Districte,Barri,Preu m2 útil % var,Preu m2 útil 2021,Preu m2 útil 2022,Preu m2 útil var,Preu mitjà % var,Preu mitjà 2021,Preu mitjà 2022,Preu mitjà var,Superfície útil % var,Superfície útil 2021,Superfície útil 2022,Superfície útil var,Unitats % var,Unitats 2021,Unitats 2022,Unitats var
0,Barcelona,NaN,NaN,4.585765,7158.685476,7486.965946,328.280470,3.788473,6.075427e+05,6.305593e+05,23016.588272,-1.541476,82.121624,80.855739,-1.265885,9.362707,1271.0,1390.0,119.0
1,Barcelona,01 Ciutat Vella,NaN,-3.145547,8401.623938,8137.346917,-264.277020,-3.565175,7.038114e+05,6.787193e+05,-25092.105263,-2.486524,83.146222,81.078771,-2.067451,50.000000,114.0,171.0,57.0
5,Barcelona,02 Eixample,NaN,5.741050,10230.460681,10817.796514,587.335833,-0.259690,1.040587e+06,1.037885e+06,-2702.300465,-6.059786,97.858322,91.928317,-5.930005,38.378378,185.0,256.0,71.0
11,Barcelona,03 Sants-Montjuïc,NaN,5.324598,5639.947792,5940.252324,300.304532,5.231332,4.582672e+05,4.822407e+05,23973.479341,0.186666,85.545284,85.704968,0.159684,-9.000000,100.0,91.0,-9.0
19,Barcelona,04 Les Corts,NaN,-10.618884,8130.131726,7266.802442,-863.329285,2.466735,9.429176e+05,9.661769e+05,23259.276018,11.313211,123.347230,137.301762,13.954532,-23.529412,34.0,26.0,-8.0
23,Barcelona,05 Sarrià - Sant Gervasi,NaN,5.502445,10209.148559,10770.901377,561.752818,6.618407,1.112140e+06,1.185746e+06,73605.979133,2.508705,107.016763,109.701498,2.684735,-5.617978,89.0,84.0,-5.0
29,Barcelona,06 Gràcia,NaN,4.247644,6952.147170,7247.449632,295.302461,3.729695,4.819200e+05,4.998941e+05,17974.143635,-0.361510,70.790666,70.534751,-0.255915,-3.676471,136.0,131.0,-5.0
34,Barcelona,07 Horta-Guinardó,NaN,6.238985,5608.421967,5958.330569,349.908602,7.974033,3.865443e+05,4.173675e+05,30823.172318,1.403981,70.122406,71.106911,0.984505,7.386364,176.0,189.0,13.0
44,Barcelona,08 Nou Barris,NaN,-3.229433,5152.412799,4986.019095,-166.393704,2.807527,3.076593e+05,3.162969e+05,8637.615741,5.083588,60.303519,63.369101,3.065582,77.777778,27.0,48.0,21.0
50,Barcelona,09 Sant Andreu,NaN,2.053706,5148.573688,5254.310279,105.736591,-4.454101,3.972671e+05,3.795724e+05,-17694.677342,-7.434342,78.190490,72.377542,-5.812949,52.054795,146.0,222.0,76.0


EXPORTAR TAULES

In [54]:

list_tables_export = ["table11","table17","table33", "table38", "table42","table76", "table80_total", "table80_uni", 
                      "table80_pluri","table101_def","table117", "table121", "table125", "table134", 
                        "table151","table190", "table205", "table208"]

writer = ExcelWriter(path+ 'Estudi_oferta_taules_1S2023.xlsx')

# Write each table to a separate sheet in the Excel file
for table in list_tables_export:
    eval(table).to_excel(writer, sheet_name=table, index=True)


writer.save()